In [1]:
import pickle
import pandas as pd
import numpy as np
import ml_Suffle_and_validation as sav
import psycopg2
import datetime
import types
import pandas.io.sql as sqlio

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from psycopg2 import sql

with open("../db_config.json", "r") as db_config:
    db_config = json.load(db_config)

dbname = db_config['redshift']['NAME']
host = db_config['redshift']['HOST'] 
port = db_config['redshift']['PORT'] 
user = db_config['redshift']['USER'] 
password= db_config['redshift']['PASSWORD'] 
connect_param = dict({'dbname':dbname, 'host':host, 'port':port, 'user':user, 'password':password})
random_state = 20

# 1. 재무 수치 데이터, 비율 데이터, 감사보고서 데이터, 주가 데이터, 부도기사 비율, 거시경제 데이터로 학습

## 1-1. Redshift에서 데이터 추출 및 Dataframe으로 가공하는 함수

In [2]:
def create_df_for_ml(prediction_range='6m', years=2):
    column_for_table_3 = ['stock_code'
    ,'Y-1_총자산(천원)'
    ,'Y-1_현금및현금성자산(천원)'
    ,'Y-1_총부채(천원)'
    ,'Y-1_총자본(천원)'
    ,'Y-1_매출액(천원)'
    ,'Y-1_당기순이익(천원)'
    ,'Y-1_세전계속사업이익(천원)'
    ,'Y-1_영업이익(천원)'
    ,'Y-1_매출총이익(천원)'
    ,'Y-1_차입금의존도(p)'
    ,'Y-1_당기순이익률(p)'
    ,'Y-1_세전계속사업이익률(p)'
    ,'Y-1_영업이익률(p)'
    ,'Y-1_매출총이익률(p)'
    ,'Y-1_총자산회전율(회)'
    ,'Y-1_총자산증가율(p)'
    ,'Y-1_총자산부채비율(p)'
    ,'Y-2_총자산(천원)'
    ,'Y-2_현금및현금성자산(천원)'
    ,'Y-2_총부채(천원)'
    ,'Y-2_총자본(천원)'
    ,'Y-2_매출액(천원)'
    ,'Y-2_당기순이익(천원)'
    ,'Y-2_세전계속사업이익(천원)'
    ,'Y-2_영업이익(천원)'
    ,'Y-2_매출총이익(천원)'
    ,'Y-2_차입금의존도(p)'
    ,'Y-2_당기순이익률(p)'
    ,'Y-2_세전계속사업이익률(p)'
    ,'Y-2_영업이익률(p)'
    ,'Y-2_매출총이익률(p)'
    ,'Y-2_총자산회전율(회)'
    ,'Y-2_총자산증가율(p)'
    ,'Y-2_총자산부채비율(p)'
    ,'Y-3_총자산(천원)'
    ,'Y-3_현금및현금성자산(천원)'
    ,'Y-3_총부채(천원)'
    ,'Y-3_총자본(천원)'
    ,'Y-3_매출액(천원)'
    ,'Y-3_당기순이익(천원)'
    ,'Y-3_세전계속사업이익(천원)'
    ,'Y-3_영업이익(천원)'
    ,'Y-3_매출총이익(천원)'
    ,'Y-3_차입금의존도(p)'
    ,'Y-3_당기순이익률(p)'
    ,'Y-3_세전계속사업이익률(p)'
    ,'Y-3_영업이익률(p)'
    ,'Y-3_매출총이익률(p)'
    ,'Y-3_총자산회전율(회)'
    ,'Y-3_총자산증가율(p)'
    ,'Y-3_총자산부채비율(p)'
    ,'R1-R2_외국인계매도비율변화'
    ,'R2-R3_외국인계매도비율변화'
    ,'R3-R4_외국인계매도비율변화'
    ,'R1-R2_외국인계매수비율변화'
    ,'R2-R3_외국인계매수비율변화'
    ,'R3-R4_외국인계매수비율변화'
    ,'R1-R2_기관계매도비율변화'
    ,'R2-R3_기관계매도비율변화'
    ,'R3-R4_기관계매도비율변화'
    ,'R1-R2_기관계매수비율변화'
    ,'R2-R3_기관계매수비율변화'
    ,'R3-R4_기관계매수비율변화'
    ,'수익률 (1개월)(p)'
    ,'수익률 (3개월)(p)'
    ,'수익률 (6개월)(p)'
    ,'부도기사비율_m1'
    ,'부도기사비율_m2'
    ,'부도기사비율_m3'
    ,'부도기사비율_m4'
    ,'부도기사비율_m5'
    ,'부도기사비율_m6'
    ,'부도기사비율_m7'
    ,'부도기사비율_m8'
    ,'부도기사비율_m9'
    ,'부도기사비율_m10'
    ,'부도기사비율_m11'
    ,'부도기사비율_m12'
    ,'n_opinion_in_r3'
    ,'m1_oil_price'
    ,'m2_oil_price'
    ,'m3_oil_price'
    ,'m4_oil_price'
    ,'m5_oil_price'
    ,'m6_oil_price'
    ,'m7_oil_price'
    ,'m8_oil_price'
    ,'m9_oil_price'
    ,'m10_oil_price'
    ,'m11_oil_price'
    ,'m12_oil_price'
    ,'m1_oil_price_roc_from_month_year_earlier'
    ,'m2_oil_price_roc_from_month_year_earlier'
    ,'m3_oil_price_roc_from_month_year_earlier'
    ,'m4_oil_price_roc_from_month_year_earlier'
    ,'m5_oil_price_roc_from_month_year_earlier'
    ,'m6_oil_price_roc_from_month_year_earlier'
    ,'m7_oil_price_roc_from_month_year_earlier'
    ,'m8_oil_price_roc_from_month_year_earlier'
    ,'m9_oil_price_roc_from_month_year_earlier'
    ,'m10_oil_price_roc_from_month_year_earlier'
    ,'m11_oil_price_roc_from_month_year_earlier'
    ,'m12_oil_price_roc_from_month_year_earlier'
    ,'m1_CD유통수익률(91일)(p)'
    ,'m2_CD유통수익률(91일)(p)'
    ,'m3_CD유통수익률(91일)(p)'
    ,'m4_CD유통수익률(91일)(p)'
    ,'m5_CD유통수익률(91일)(p)'
    ,'m6_CD유통수익률(91일)(p)'
    ,'m7_CD유통수익률(91일)(p)'
    ,'m8_CD유통수익률(91일)(p)'
    ,'m9_CD유통수익률(91일)(p)'
    ,'m10_CD유통수익률(91일)(p)'
    ,'m11_CD유통수익률(91일)(p)'
    ,'m12_CD유통수익률(91일)(p)'
    ,'m1_국고채(3년)(p)'
    ,'m2_국고채(3년)(p)'
    ,'m3_국고채(3년)(p)'
    ,'m4_국고채(3년)(p)'
    ,'m5_국고채(3년)(p)'
    ,'m6_국고채(3년)(p)'
    ,'m7_국고채(3년)(p)'
    ,'m8_국고채(3년)(p)'
    ,'m9_국고채(3년)(p)'
    ,'m10_국고채(3년)(p)'
    ,'m11_국고채(3년)(p)'
    ,'m12_국고채(3년)(p)'
    ,'m1_원달러환율(매매기준율)'
    ,'m2_원달러환율(매매기준율)'
    ,'m3_원달러환율(매매기준율)'
    ,'m4_원달러환율(매매기준율)'
    ,'m5_원달러환율(매매기준율)'
    ,'m6_원달러환율(매매기준율)'
    ,'m7_원달러환율(매매기준율)'
    ,'m8_원달러환율(매매기준율)'
    ,'m9_원달러환율(매매기준율)'
    ,'m10_원달러환율(매매기준율)'
    ,'m11_원달러환율(매매기준율)'
    ,'m12_원달러환율(매매기준율)'
    ,'y1_gdp_growth_rate'
    ,'y2_gdp_growth_rate'
    ,'y3_gdp_growth_rate'
    ,'bankruptcy'
    ]
    
    column_for_table_2 = ['stock_code'
    ,'Y-1_총자산(천원)'
    ,'Y-1_현금및현금성자산(천원)'
    ,'Y-1_총부채(천원)'
    ,'Y-1_총자본(천원)'
    ,'Y-1_매출액(천원)'
    ,'Y-1_당기순이익(천원)'
    ,'Y-1_세전계속사업이익(천원)'
    ,'Y-1_영업이익(천원)'
    ,'Y-1_매출총이익(천원)'
    ,'Y-1_차입금의존도(p)'
    ,'Y-1_당기순이익률(p)'
    ,'Y-1_세전계속사업이익률(p)'
    ,'Y-1_영업이익률(p)'
    ,'Y-1_매출총이익률(p)'
    ,'Y-1_총자산회전율(회)'
    ,'Y-1_총자산증가율(p)'
    ,'Y-1_총자산부채비율(p)'
    ,'Y-2_총자산(천원)'
    ,'Y-2_현금및현금성자산(천원)'
    ,'Y-2_총부채(천원)'
    ,'Y-2_총자본(천원)'
    ,'Y-2_매출액(천원)'
    ,'Y-2_당기순이익(천원)'
    ,'Y-2_세전계속사업이익(천원)'
    ,'Y-2_영업이익(천원)'
    ,'Y-2_매출총이익(천원)'
    ,'Y-2_차입금의존도(p)'
    ,'Y-2_당기순이익률(p)'
    ,'Y-2_세전계속사업이익률(p)'
    ,'Y-2_영업이익률(p)'
    ,'Y-2_매출총이익률(p)'
    ,'Y-2_총자산회전율(회)'
    ,'Y-2_총자산증가율(p)'
    ,'Y-2_총자산부채비율(p)'
    ,'R1-R2_외국인계매도비율변화'
    ,'R2-R3_외국인계매도비율변화'
    ,'R3-R4_외국인계매도비율변화'
    ,'R1-R2_외국인계매수비율변화'
    ,'R2-R3_외국인계매수비율변화'
    ,'R3-R4_외국인계매수비율변화'
    ,'R1-R2_기관계매도비율변화'
    ,'R2-R3_기관계매도비율변화'
    ,'R3-R4_기관계매도비율변화'
    ,'R1-R2_기관계매수비율변화'
    ,'R2-R3_기관계매수비율변화'
    ,'R3-R4_기관계매수비율변화'
    ,'수익률 (1개월)(p)'
    ,'수익률 (3개월)(p)'
    ,'수익률 (6개월)(p)'
    ,'부도기사비율_m1'
    ,'부도기사비율_m2'
    ,'부도기사비율_m3'
    ,'부도기사비율_m4'
    ,'부도기사비율_m5'
    ,'부도기사비율_m6'
    ,'부도기사비율_m7'
    ,'부도기사비율_m8'
    ,'부도기사비율_m9'
    ,'부도기사비율_m10'
    ,'부도기사비율_m11'
    ,'부도기사비율_m12'
    ,'n_opinion_in_r3'
    ,'m1_oil_price'
    ,'m2_oil_price'
    ,'m3_oil_price'
    ,'m4_oil_price'
    ,'m5_oil_price'
    ,'m6_oil_price'
    ,'m7_oil_price'
    ,'m8_oil_price'
    ,'m9_oil_price'
    ,'m10_oil_price'
    ,'m11_oil_price'
    ,'m12_oil_price'
    ,'m1_oil_price_roc_from_month_year_earlier'
    ,'m2_oil_price_roc_from_month_year_earlier'
    ,'m3_oil_price_roc_from_month_year_earlier'
    ,'m4_oil_price_roc_from_month_year_earlier'
    ,'m5_oil_price_roc_from_month_year_earlier'
    ,'m6_oil_price_roc_from_month_year_earlier'
    ,'m7_oil_price_roc_from_month_year_earlier'
    ,'m8_oil_price_roc_from_month_year_earlier'
    ,'m9_oil_price_roc_from_month_year_earlier'
    ,'m10_oil_price_roc_from_month_year_earlier'
    ,'m11_oil_price_roc_from_month_year_earlier'
    ,'m12_oil_price_roc_from_month_year_earlier'
    ,'m1_CD유통수익률(91일)(p)'
    ,'m2_CD유통수익률(91일)(p)'
    ,'m3_CD유통수익률(91일)(p)'
    ,'m4_CD유통수익률(91일)(p)'
    ,'m5_CD유통수익률(91일)(p)'
    ,'m6_CD유통수익률(91일)(p)'
    ,'m7_CD유통수익률(91일)(p)'
    ,'m8_CD유통수익률(91일)(p)'
    ,'m9_CD유통수익률(91일)(p)'
    ,'m10_CD유통수익률(91일)(p)'
    ,'m11_CD유통수익률(91일)(p)'
    ,'m12_CD유통수익률(91일)(p)'
    ,'m1_국고채(3년)(p)'
    ,'m2_국고채(3년)(p)'
    ,'m3_국고채(3년)(p)'
    ,'m4_국고채(3년)(p)'
    ,'m5_국고채(3년)(p)'
    ,'m6_국고채(3년)(p)'
    ,'m7_국고채(3년)(p)'
    ,'m8_국고채(3년)(p)'
    ,'m9_국고채(3년)(p)'
    ,'m10_국고채(3년)(p)'
    ,'m11_국고채(3년)(p)'
    ,'m12_국고채(3년)(p)'
    ,'m1_원달러환율(매매기준율)'
    ,'m2_원달러환율(매매기준율)'
    ,'m3_원달러환율(매매기준율)'
    ,'m4_원달러환율(매매기준율)'
    ,'m5_원달러환율(매매기준율)'
    ,'m6_원달러환율(매매기준율)'
    ,'m7_원달러환율(매매기준율)'
    ,'m8_원달러환율(매매기준율)'
    ,'m9_원달러환율(매매기준율)'
    ,'m10_원달러환율(매매기준율)'
    ,'m11_원달러환율(매매기준율)'
    ,'m12_원달러환율(매매기준율)'
    ,'y1_gdp_growth_rate'
    ,'y2_gdp_growth_rate'
    ,'y3_gdp_growth_rate'
    ,'bankruptcy'
    ]
    
    if years == 3:
        column_for_table_cache = column_for_table_3
    elif years == 2:
        column_for_table_cache = column_for_table_2
        
    column_for_table_dict = {}
    for each in column_for_table_cache:
        column_for_table_dict[each] = []
    df_for_learning = pd.DataFrame(column_for_table_dict)

    # 주가데이터 가공
    with psycopg2.connect(**connect_param) as con:
        with con.cursor() as cur:
            stock_table = 'stock_data_3years_raw_' + prediction_range
            pre_date = 'pre_'+prediction_range
            
            # 외국인/기관계 매도, 매수비율 임시테이블 제작
            temp_table_create_query_list = [
                sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R1_외국인계매도비율"
                INTO TEMPORARY "temp_R1_외국인계매도비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < comp_list.{pre_date_1} AND 
                      stock_data.date > (comp_list.{pre_date_2} - 30) AND 
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R2_외국인계매도비율"
                INTO TEMPORARY "temp_R2_외국인계매도비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 31) AND
                      stock_data.date > (comp_list.{pre_date_2} - 60) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R3_외국인계매도비율"
                INTO TEMPORARY "temp_R3_외국인계매도비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 61) AND 
                      stock_data.date > (comp_list.{pre_date_2} - 90) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R4_외국인계매도비율"
                INTO TEMPORARY "temp_R4_외국인계매도비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 91) AND
                      stock_data.date > (comp_list.{pre_date_2} - 120) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R1_외국인계매수비율"
                INTO TEMPORARY "temp_R1_외국인계매수비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < comp_list.{pre_date_1} AND
                      stock_data.date > (comp_list.{pre_date_2} - 30) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R2_외국인계매수비율"
                INTO TEMPORARY "temp_R2_외국인계매수비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 31) AND
                      stock_data.date > (comp_list.{pre_date_2} - 60) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R3_외국인계매수비율"
                INTO TEMPORARY "temp_R3_외국인계매수비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 61) AND
                      stock_data.date > (comp_list.{pre_date_2} - 90) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R4_외국인계매수비율"
                INTO TEMPORARY "temp_R4_외국인계매수비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 91) AND 
                      stock_data.date > (comp_list.{pre_date_2} - 120) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R1_기관계매도비율"
                INTO TEMPORARY "temp_R1_기관계매도비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < comp_list.{pre_date_1} AND
                      stock_data.date > (comp_list.{pre_date_2} - 30) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R2_기관계매도비율"
                INTO TEMPORARY "temp_R2_기관계매도비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 31) AND
                      stock_data.date > (comp_list.{pre_date_2} - 60) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R3_기관계매도비율"
                INTO TEMPORARY "temp_R3_기관계매도비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 61) AND
                      stock_data.date > (comp_list.{pre_date_2} - 90) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R4_기관계매도비율"
                INTO TEMPORARY "temp_R4_기관계매도비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 91) AND
                      stock_data.date > (comp_list.{pre_date_2} - 120) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R1_기관계매수비율"
                INTO TEMPORARY "temp_R1_기관계매수비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < comp_list.{pre_date_1} AND
                      stock_data.date > (comp_list.{pre_date_2} - 30) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R2_기관계매수비율"
                INTO TEMPORARY "temp_R2_기관계매수비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 31) AND
                      stock_data.date > (comp_list.{pre_date_2} - 60) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R3_기관계매수비율"
                INTO TEMPORARY "temp_R3_기관계매수비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 61) AND
                      stock_data.date > (comp_list.{pre_date_2} - 90) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date)),
                 sql.SQL("""SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R4_기관계매수비율"
                INTO TEMPORARY "temp_R4_기관계매수비율"
                FROM {table_name} stock_data INNER JOIN target_company_list comp_list ON stock_data.stock_code = comp_list.stock_code
                WHERE stock_data.date < (comp_list.{pre_date_1} - 91) AND
                      stock_data.date > (comp_list.{pre_date_2} - 120) AND
                      stock_data."종가(원)" IS NOT NULL
                GROUP BY stock_data.stock_code;""").format(table_name = sql.Identifier(stock_table), pre_date_1 = sql.Identifier(pre_date), pre_date_2 = sql.Identifier(pre_date))
            ]

            for each_query in temp_table_create_query_list:
                cur.execute(each_query)
            con.commit()
            print('create_temp_table')

            # 예측 주가데이터 테이블 만들기
            table_for_dt_based_model_stock_query = """SELECT 
                r1_fs.stock_code
                ,r1_fs."R1_외국인계매도비율" - r2_fs."R2_외국인계매도비율" AS "R1-R2_외국인계매도비율변화"
                ,r2_fs."R2_외국인계매도비율" - r3_fs."R3_외국인계매도비율" AS "R2-R3_외국인계매도비율변화"
                ,r3_fs."R3_외국인계매도비율" - r4_fs."R4_외국인계매도비율" AS "R3-R4_외국인계매도비율변화"
                ,r1_fb."R1_외국인계매수비율" - r2_fb."R2_외국인계매수비율" AS "R1-R2_외국인계매수비율변화"
                ,r2_fb."R2_외국인계매수비율" - r3_fb."R3_외국인계매수비율" AS "R2-R3_외국인계매수비율변화"
                ,r3_fb."R3_외국인계매수비율" - r4_fb."R4_외국인계매수비율" AS "R3-R4_외국인계매수비율변화"
                ,r1_os."R1_기관계매도비율" - r2_os."R2_기관계매도비율" AS "R1-R2_기관계매도비율변화"
                ,r2_os."R2_기관계매도비율" - r3_os."R3_기관계매도비율" AS "R2-R3_기관계매도비율변화"
                ,r3_os."R3_기관계매도비율" - r4_os."R4_기관계매도비율" AS "R3-R4_기관계매도비율변화"
                ,r1_ob."R1_기관계매수비율" - r2_ob."R2_기관계매수비율" AS "R1-R2_기관계매수비율변화"
                ,r2_ob."R2_기관계매수비율" - r3_ob."R3_기관계매수비율" AS "R2-R3_기관계매수비율변화"
                ,r3_ob."R3_기관계매수비율" - r4_ob."R4_기관계매수비율" AS "R3-R4_기관계매수비율변화"
            INTO TEMPORARY table_for_dt_based_model_stock
            FROM "temp_R1_외국인계매도비율" r1_fs INNER JOIN "temp_R2_외국인계매도비율" r2_fs ON r1_fs.stock_code = r2_fs.stock_code
            INNER JOIN "temp_R3_외국인계매도비율" r3_fs ON r1_fs.stock_code = r3_fs.stock_code
            INNER JOIN "temp_R4_외국인계매도비율" r4_fs ON r1_fs.stock_code = r4_fs.stock_code
            INNER JOIN "temp_R1_외국인계매수비율" r1_fb ON r1_fs.stock_code = r1_fb.stock_code
            INNER JOIN "temp_R2_외국인계매수비율" r2_fb ON r1_fs.stock_code = r2_fb.stock_code
            INNER JOIN "temp_R3_외국인계매수비율" r3_fb ON r1_fs.stock_code = r3_fb.stock_code
            INNER JOIN "temp_R4_외국인계매수비율" r4_fb ON r1_fs.stock_code = r4_fb.stock_code
            INNER JOIN "temp_R1_기관계매도비율" r1_os ON r1_fs.stock_code = r1_os.stock_code
            INNER JOIN "temp_R2_기관계매도비율" r2_os ON r1_fs.stock_code = r2_os.stock_code
            INNER JOIN "temp_R3_기관계매도비율" r3_os ON r1_fs.stock_code = r3_os.stock_code
            INNER JOIN "temp_R4_기관계매도비율" r4_os ON r1_fs.stock_code = r4_os.stock_code
            INNER JOIN "temp_R1_기관계매수비율" r1_ob ON r1_fs.stock_code = r1_ob.stock_code
            INNER JOIN "temp_R2_기관계매수비율" r2_ob ON r1_fs.stock_code = r2_ob.stock_code
            INNER JOIN "temp_R3_기관계매수비율" r3_ob ON r1_fs.stock_code = r3_ob.stock_code
            INNER JOIN "temp_R4_기관계매수비율" r4_ob ON r1_fs.stock_code = r4_ob.stock_code;"""

            cur.execute(table_for_dt_based_model_stock_query)
            con.commit()
            print('create_table_for_dt_based_model_stock')

            # 주가 수익률 임시테이블 만들기
            temp_earning_rate_query = sql.SQL("""
            select stock.stock_code, stock."수익률 (1개월)(p)", stock."수익률 (3개월)(p)", stock."수익률 (6개월)(p)"
            INTO TEMPORARY temp_earning_rate
            from {table_name} stock LEFT OUTER JOIN target_company_list target ON stock.stock_code = target.stock_code
            where stock.date = (target.{pre_date} - 1);""").format(table_name=sql.Identifier(stock_table), pre_date = sql.Identifier(pre_date))

            cur.execute(temp_earning_rate_query)
            con.commit()
            print('create_temp_earning_rate_query')

            # 감사보고서 임시테이블 만들기
            temp_audit_report_query = sql.SQL("""
            SELECT audit_cache.stock_code AS stock_code
            ,CASE WHEN (MOD(sum(CASE WHEN audit_cache.n_opinion = TRUE THEN 1 WHEN audit_cache.n_opinion = FALSE THEN 0 WHEN audit_cache.n_opinion IS NULL THEN 100 END), 100)) != 0 THEN True
                  WHEN ((MOD(sum(CASE WHEN audit_cache.n_opinion = TRUE THEN 1 WHEN audit_cache.n_opinion = FALSE THEN 0 WHEN audit_cache.n_opinion IS NULL THEN 100 END), 100) = 0) 
                      AND (sum(CASE WHEN audit_cache.n_opinion = TRUE THEN 1 WHEN audit_cache.n_opinion = FALSE THEN 0 WHEN audit_cache.n_opinion IS NULL THEN 100 END)) < 100) THEN False
                  WHEN ((MOD(sum(CASE WHEN audit_cache.n_opinion = TRUE THEN 1 WHEN audit_cache.n_opinion = FALSE THEN 0 WHEN audit_cache.n_opinion IS NULL THEN 100 END), 100) = 0)
                      AND (sum(CASE WHEN audit_cache.n_opinion = TRUE THEN 1 WHEN audit_cache.n_opinion = FALSE THEN 0 WHEN audit_cache.n_opinion IS NULL THEN 100 END)) >= 100) THEN False
             END AS n_opinion_in_r3
            INTO TEMPORARY "temp_audit_report_table"
            FROM (
                SELECT audit.stock_code
                    ,audit.report_date
                    ,audit.n_opinion
                    ,row_number() OVER (PARTITION BY audit.stock_code ORDER BY audit.report_date DESC NULLS LAST) AS rn
                FROM dart_audit_report_data audit INNER JOIN target_company_list target ON audit.stock_code = target.stock_code
                WHERE (audit.report_date IS NULL) OR (audit.report_date < target.{pre_date})) AS audit_cache
            WHERE audit_cache.rn < 4
            GROUP BY audit_cache.stock_code;
            """).format(pre_date = sql.Identifier(pre_date))
            cur.execute(temp_audit_report_query)
            con.commit()
            print('create_temp_audit_report_query')

            # 학습용 테이블 select
            final_select_query = sql.SQL("""select
            fi.*
            ,stock."R1-R2_외국인계매도비율변화"
            ,stock."R2-R3_외국인계매도비율변화"
            ,stock."R3-R4_외국인계매도비율변화"
            ,stock."R1-R2_외국인계매수비율변화"
            ,stock."R2-R3_외국인계매수비율변화"
            ,stock."R3-R4_외국인계매수비율변화"
            ,stock."R1-R2_기관계매도비율변화"
            ,stock."R2-R3_기관계매도비율변화"
            ,stock."R3-R4_기관계매도비율변화"
            ,stock."R1-R2_기관계매수비율변화"
            ,stock."R2-R3_기관계매수비율변화"
            ,stock."R3-R4_기관계매수비율변화"
            ,earn_r."수익률 (1개월)(p)"
            ,earn_r."수익률 (3개월)(p)"
            ,earn_r."수익률 (6개월)(p)"
            ,article_r.m1
            ,article_r.m2
            ,article_r.m3
            ,article_r.m4
            ,article_r.m5
            ,article_r.m6
            ,article_r.m7
            ,article_r.m8
            ,article_r.m9
            ,article_r.m10
            ,article_r.m11
            ,article_r.m12
            ,audit.n_opinion_in_r3
            ,macroeco."m1_oil_price"
            ,macroeco."m2_oil_price"
            ,macroeco."m3_oil_price"
            ,macroeco."m4_oil_price"
            ,macroeco."m5_oil_price"
            ,macroeco."m6_oil_price"
            ,macroeco."m7_oil_price"
            ,macroeco."m8_oil_price"
            ,macroeco."m9_oil_price"
            ,macroeco."m10_oil_price"
            ,macroeco."m11_oil_price"
            ,macroeco."m12_oil_price"
            ,macroeco."m1_oil_price_roc_from_month_year_earlier"
            ,macroeco."m2_oil_price_roc_from_month_year_earlier"
            ,macroeco."m3_oil_price_roc_from_month_year_earlier"
            ,macroeco."m4_oil_price_roc_from_month_year_earlier"
            ,macroeco."m5_oil_price_roc_from_month_year_earlier"
            ,macroeco."m6_oil_price_roc_from_month_year_earlier"
            ,macroeco."m7_oil_price_roc_from_month_year_earlier"
            ,macroeco."m8_oil_price_roc_from_month_year_earlier"
            ,macroeco."m9_oil_price_roc_from_month_year_earlier"
            ,macroeco."m10_oil_price_roc_from_month_year_earlier"
            ,macroeco."m11_oil_price_roc_from_month_year_earlier"
            ,macroeco."m12_oil_price_roc_from_month_year_earlier"
            ,macroeco."m1_CD유통수익률(91일)(p)"
            ,macroeco."m2_CD유통수익률(91일)(p)"
            ,macroeco."m3_CD유통수익률(91일)(p)"
            ,macroeco."m4_CD유통수익률(91일)(p)"
            ,macroeco."m5_CD유통수익률(91일)(p)"
            ,macroeco."m6_CD유통수익률(91일)(p)"
            ,macroeco."m7_CD유통수익률(91일)(p)"
            ,macroeco."m8_CD유통수익률(91일)(p)"
            ,macroeco."m9_CD유통수익률(91일)(p)"
            ,macroeco."m10_CD유통수익률(91일)(p)"
            ,macroeco."m11_CD유통수익률(91일)(p)"
            ,macroeco."m12_CD유통수익률(91일)(p)"
            ,macroeco."m1_국고채(3년)(p)"
            ,macroeco."m2_국고채(3년)(p)"
            ,macroeco."m3_국고채(3년)(p)"
            ,macroeco."m4_국고채(3년)(p)"
            ,macroeco."m5_국고채(3년)(p)"
            ,macroeco."m6_국고채(3년)(p)"
            ,macroeco."m7_국고채(3년)(p)"
            ,macroeco."m8_국고채(3년)(p)"
            ,macroeco."m9_국고채(3년)(p)"
            ,macroeco."m10_국고채(3년)(p)"
            ,macroeco."m11_국고채(3년)(p)"
            ,macroeco."m12_국고채(3년)(p)"
            ,macroeco."m1_원달러환율(매매기준율)"
            ,macroeco."m2_원달러환율(매매기준율)"
            ,macroeco."m3_원달러환율(매매기준율)"
            ,macroeco."m4_원달러환율(매매기준율)"
            ,macroeco."m5_원달러환율(매매기준율)"
            ,macroeco."m6_원달러환율(매매기준율)"
            ,macroeco."m7_원달러환율(매매기준율)"
            ,macroeco."m8_원달러환율(매매기준율)"
            ,macroeco."m9_원달러환율(매매기준율)"
            ,macroeco."m10_원달러환율(매매기준율)"
            ,macroeco."m11_원달러환율(매매기준율)"
            ,macroeco."m12_원달러환율(매매기준율)"
            ,macroeco."y1_gdp_growth_rate"
            ,macroeco."y2_gdp_growth_rate"
            ,macroeco."y3_gdp_growth_rate"
            ,target.bankruptcy
            from {table_name} fi
                INNER JOIN table_for_dt_based_model_stock stock ON fi.stock_code = stock.stock_code
                INNER JOIN temp_earning_rate earn_r ON fi.stock_code = earn_r.stock_code
                INNER JOIN temp_audit_report_table audit ON fi.stock_code = audit.stock_code
                INNER JOIN target_company_list target ON fi.stock_code = target.stock_code
                INNER JOIN {table_name_article} article_r ON fi.stock_code = article_r.stock_code
                INNER JOIN {table_name_macroeco} macroeco ON fi.stock_code = macroeco.stock_code;""")\
            .format(table_name = sql.Identifier('table_for_dt_based_model_'+prediction_range+'_finance_'+str(years)+'y')
                ,table_name_article = sql.Identifier('bankruptcy_article_ratio_table_'+prediction_range)
                ,table_name_macroeco = sql.Identifier('table_for_dt_based_model_'+prediction_range+'_macroeco')
            )

            cur.execute(final_select_query)
            print('final_select_query')

            for each_row in cur:
                new_row = {}
                for idx, each_value in enumerate(each_row):
                    new_row[column_for_table_cache[idx]]=each_value
                #print(new_row)
                df_for_learning = df_for_learning.append(new_row, ignore_index=True)
            #print(df_for_learning['n_opinion_in_r3'])
            df_for_learning.set_index('stock_code', inplace=True)
            df_for_learning['bankruptcy'] = df_for_learning['bankruptcy'].apply(lambda x : False if x == 0.0 else True)
            df_for_learning['n_opinion_in_r3'] = df_for_learning['n_opinion_in_r3'].apply(lambda x : False if x == 0.0 else True)
            df_for_learning.dropna(inplace=True)
            df_for_learning = df_for_learning.astype({"수익률 (6개월)(p)": float}, copy=False)

            X = df_for_learning.drop(columns=['bankruptcy'])
            y = df_for_learning['bankruptcy']
    return (X,y)

## 1-2. Random Forest, GradientBoosting, XGboost 학습 - 6개월 예측

In [3]:
pre6m_fi2y = create_df_for_ml(prediction_range='6m', years=2)
X_6m = pre6m_fi2y[0]
y_6m = pre6m_fi2y[1]

create_temp_table
create_table_for_dt_based_model_stock
create_temp_earning_rate_query
create_temp_audit_report_query
final_select_query


In [4]:
X_6m

,Y-1_총자산(천원),Y-1_현금및현금성자산(천원),Y-1_총부채(천원),Y-1_총자본(천원),Y-1_매출액(천원),Y-1_당기순이익(천원),Y-1_세전계속사업이익(천원),Y-1_영업이익(천원),Y-1_매출총이익(천원),Y-1_차입금의존도(p),...,m6_원달러환율(매매기준율),m7_원달러환율(매매기준율),m8_원달러환율(매매기준율),m9_원달러환율(매매기준율),m10_원달러환율(매매기준율),m11_원달러환율(매매기준율),m12_원달러환율(매매기준율),y1_gdp_growth_rate,y2_gdp_growth_rate,y3_gdp_growth_rate
stock_code,,,,,,,,,,,,,,,,,,,,,
008670,2.887228e+09,1.599800e+07,2.197262e+09,6.899660e+08,2.811410e+08,30311000.0,37065000.0,42307000.0,2.811410e+08,14.02,...,1166.34,1166.69,1184.32,1192.95,1184.87,1166.27,1166.17,0.031,0.077,0.049
035910,3.869358e+07,4.498010e+05,4.206940e+07,-3.375816e+06,5.612976e+07,-15379698.0,-15379698.0,-8882955.0,3.290716e+06,90.42,...,1150.85,1166.34,1166.69,1184.32,1192.95,1184.87,1166.27,0.031,0.077,0.049
037640,4.960932e+07,2.595210e+05,4.305133e+07,6.557992e+06,5.162583e+07,469886.0,774972.0,2678560.0,4.504766e+06,51.50,...,1150.85,1166.34,1166.69,1184.32,1192.95,1184.87,1166.27,0.031,0.077,0.049
007910,9.845974e+07,2.198753e+06,8.763419e+06,8.969632e+07,6.246431e+07,7345991.0,9996433.0,527409.0,8.468384e+06,0.00,...,1177.37,1150.85,1166.34,1166.69,1184.32,1192.95,1184.87,0.031,0.077,0.049
054080,6.488546e+07,8.974588e+06,3.043291e+07,3.445255e+07,3.036421e+07,2680136.0,2927240.0,3379793.0,9.497963e+06,42.46,...,1158.65,1177.37,1150.85,1166.34,1166.69,1184.32,1192.95,0.031,0.077,0.049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
000210,1.283372e+10,2.134502e+09,6.783473e+09,6.050244e+09,1.098449e+10,678088774.0,894440698.0,845358146.0,1.420162e+09,20.72,...,1130.72,1122.45,1122.00,1122.90,1128.58,1130.81,1120.60,0.032,0.029,0.028
000140,3.989286e+09,3.096063e+08,2.893500e+09,1.095787e+09,1.876650e+09,8640980.0,29317344.0,103511877.0,7.671412e+08,42.37,...,1130.72,1122.45,1122.00,1122.90,1128.58,1130.81,1120.60,0.032,0.029,0.028
000070,3.313797e+09,1.032906e+08,1.311855e+09,2.001942e+09,2.563476e+09,102512252.0,129309979.0,138458000.0,4.821060e+08,23.50,...,1130.72,1122.45,1122.00,1122.90,1128.58,1130.81,1120.60,0.032,0.029,0.028


In [5]:
y_6m

stock_code
008670    False
035910     True
037640     True
007910    False
054080     True
          ...  
000210    False
000140    False
000070    False
000050    False
000020    False
Name: bankruptcy, Length: 2526, dtype: bool

In [6]:
sum(y_6m) / len(y_6m) # 부도기업 비율

0.14330958036421218

In [7]:
RandomForest_clf = RandomForestClassifier(random_state=random_state, n_jobs=-1)
GradientBoosting_clf = GradientBoostingClassifier(random_state=random_state)
XGBoost_clf = XGBClassifier(random_state=random_state, n_jobs=-1)

In [8]:
rand_df_6m_nonsample = sav.suffle_and_validation(X_6m, y_6m, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25)
rand_df_6m_rand_RandomOverSampling = sav.suffle_and_validation(X_6m, y_6m, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='RandomOverSampling')
rand_df_6m_rand_ADASYN = sav.suffle_and_validation(X_6m, y_6m, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='ADASYN')
rand_df_6m_rand_SMOTE = sav.suffle_and_validation(X_6m, y_6m, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='SMOTE')
GradientBoosting_df_6m_nonsample = sav.suffle_and_validation(X_6m, y_6m, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25)
GradientBoosting_df_6m_RandomOverSampling = sav.suffle_and_validation(X_6m, y_6m, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='RandomOverSampling')
GradientBoosting_df_6m_ADASYN = sav.suffle_and_validation(X_6m, y_6m, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='ADASYN')
GradientBoosting_df_6m_SMOTE = sav.suffle_and_validation(X_6m, y_6m, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='SMOTE')
XGboost_df_6m_nonsample = sav.suffle_and_validation(X_6m, y_6m, XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25)
XGboost_df_6m_RandomOverSampling = sav.suffle_and_validation(X_6m, y_6m, XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='RandomOverSampling')
XGboost_df_6m_ADASYN = sav.suffle_and_validation(X_6m, y_6m,XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='ADASYN')
XGboost_df_6m_SMOTE = sav.suffle_and_validation(X_6m, y_6m, XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='SMOTE')

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
30회 최종평균결과
Train
생존 | precision : 1.0, recall : 1.0, f1-score:1.0
부도 | precision : 1.0, recall : 1.0, f1-score:1.0
accuracy : 1.0
Test
생존 | precision : 0.988, recall : 0.981, f1-score:0.984
부도 | precision : 0.892, recall : 0.93, f1-score:0.91
accuracy : 0.973
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
30회 최종평균결과_RandomOverSampling
Train
생존 | precision : 1.0, recall : 1.0, f1-score:1.0
부도 | precision : 1.0, recall : 1.0, f1-score:1.0
accuracy : 1.0
Test
생존 | precision : 0.99, recall : 0.977, f1-score:0.984
부도 | precision : 0.876, recall : 0.941, f1-score:0.907
accuracy : 0.972
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
30회 최종평균결과_ADASYN
Train
생존 | precision : 1.0, recall : 1.0, f1-score:1.0
부도 | precision : 1.0, recall : 1.0, f1-score:1.0
accuracy : 1.0
Test
생존 | precision : 0.992, recall : 0.972, f1-score:0.982
부도 | precision : 0.852, recall : 0.952, f1-score:0.899
accuracy : 0.969
<class 'sklearn.e

* 가장 학습결과가 좋은 조합 (Test 데이터 셋 f1-score 기준) : XGboost_df_6m_nonsample

XGboost<br>
30회 최종평균결과<br>
Train<br>
생존 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
부도 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
accuracy : 1.0<br>
Test<br>
생존 | precision : 0.986, recall : 0.984, f1-score:0.985<br>
부도 | precision : 0.909, recall : 0.919, f1-score:0.913<br>
accuracy : 0.975<br>

## 1-2-1. Hyperparameter Tuning

### 1-2-1-1. max_depth, min_child_weight 조정

* n_estimator = 1000 으로 고정
* learning_rate = 0.1 로 고정

In [102]:
xgb_param_grid_6m_1 = {'max_depth': [4, 5, 6, 7]
                       ,'min_child_weight':[1, 2, 3, 4, 5]
                      }

hr_grid_6m = GridSearchCV(estimator = \
                          XGBClassifier(n_estimator = 1000, learning_rate = 0.1, random_state=random_state, n_jobs=-1,),
                       param_grid = xgb_param_grid_6m_1,
                       scoring = 'f1',
                       n_jobs = -1,
                       cv = 4,
                       refit = False, 
                       return_train_score = True)

hr_grid_6m.fit(X_6m, y_6m)
hr_grid_df_6m = pd.DataFrame(hr_grid_6m.cv_results_)
hr_grid_df_6m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
9,"{'max_depth': 5, 'min_child_weight': 5}",0.901976,0.992655
4,"{'max_depth': 4, 'min_child_weight': 5}",0.898863,0.992654
18,"{'max_depth': 7, 'min_child_weight': 4}",0.897169,0.997238
19,"{'max_depth': 7, 'min_child_weight': 5}",0.896411,0.993111
14,"{'max_depth': 6, 'min_child_weight': 5}",0.896411,0.993111
8,"{'max_depth': 5, 'min_child_weight': 4}",0.896079,0.997238
1,"{'max_depth': 4, 'min_child_weight': 2}",0.895882,0.998615
13,"{'max_depth': 6, 'min_child_weight': 4}",0.894669,0.996775
16,"{'max_depth': 7, 'min_child_weight': 2}",0.893888,0.998615
0,"{'max_depth': 4, 'min_child_weight': 1}",0.893778,1.000000


* 둘 다 5일 때 가장 좋은 값을 보임.
* min_child_weight의 값을 더 늘려 보자.

In [103]:
xgb_param_grid_6m_2 = {'min_child_weight':[6,8,10,12]}

hr_grid_6m = GridSearchCV(estimator = \
                              XGBClassifier(
                                            n_estimator = 1000
                                            ,learning_rate = 0.1
                                            ,max_depth=4
                                            ,random_state=random_state
                                            ,n_jobs=-1,
                                           ),
                       param_grid = xgb_param_grid_6m_2,
                       scoring = 'f1',
                       n_jobs = -1,
                       cv = 4,
                       refit = False, 
                       return_train_score = True)

hr_grid_6m.fit(X_6m, y_6m)
hr_grid_df_6m = pd.DataFrame(hr_grid_6m.cv_results_)
hr_grid_df_6m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
1,{'min_child_weight': 8},0.906949,0.973851
0,{'min_child_weight': 6},0.904668,0.987640
2,{'min_child_weight': 10},0.903241,0.967482
3,{'min_child_weight': 12},0.901292,0.958917


* min_child_weight는 8로 고정

### 1-2-1-2. gamma 조정

In [104]:
xgb_param_grid_6m_2 = {'gamma':[i/10.0 for i in range(0,5)]}

hr_grid_6m = GridSearchCV(estimator = \
                              XGBClassifier(
                                            n_estimator = 1000
                                            ,learning_rate = 0.1
                                            ,max_depth=4
                                            ,min_child_weight=8
                                            ,random_state=random_state
                                            ,n_jobs=-1,
                                           ),
                       param_grid = xgb_param_grid_6m_2,
                       scoring = 'f1',
                       n_jobs = -1,
                       cv = 4,
                       refit = False, 
                       return_train_score = True)

hr_grid_6m.fit(X_6m, y_6m)
hr_grid_df_6m = pd.DataFrame(hr_grid_6m.cv_results_)
hr_grid_df_6m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
0,{'gamma': 0.0},0.906949,0.973851
2,{'gamma': 0.2},0.906930,0.975239
3,{'gamma': 0.3},0.906930,0.973851
1,{'gamma': 0.1},0.906318,0.974343
4,{'gamma': 0.4},0.903741,0.976621


* gamma는 0으로 고정

### 1-2-1-3. subsample, colsample_bytree 조정

In [105]:
xgb_param_grid_6m_3 = {'subsample':[i/10.0 for i in range(6,10)],
                        'colsample_bytree':[i/10.0 for i in range(6,10)]}

hr_grid_6m = GridSearchCV(estimator = \
                              XGBClassifier(
                                            n_estimator = 1000
                                            ,learning_rate = 0.1
                                            ,max_depth=4
                                            ,min_child_weight=8
                                            ,gamma=0
                                            ,random_state=random_state
                                            ,n_jobs=-1,
                                           ),
                       param_grid = xgb_param_grid_6m_3,
                       scoring = 'f1',
                       n_jobs = -1,
                       cv = 4,
                       refit = False, 
                       return_train_score = True)

hr_grid_6m.fit(X_6m, y_6m)
hr_grid_df_6m = pd.DataFrame(hr_grid_6m.cv_results_)
hr_grid_df_6m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
1,"{'colsample_bytree': 0.6, 'subsample': 0.7}",0.906031,0.959275
5,"{'colsample_bytree': 0.7, 'subsample': 0.7}",0.904197,0.961633
14,"{'colsample_bytree': 0.9, 'subsample': 0.8}",0.904111,0.966567
9,"{'colsample_bytree': 0.8, 'subsample': 0.7}",0.903522,0.959257
13,"{'colsample_bytree': 0.9, 'subsample': 0.7}",0.902517,0.957964
0,"{'colsample_bytree': 0.6, 'subsample': 0.6}",0.902467,0.952437
10,"{'colsample_bytree': 0.8, 'subsample': 0.8}",0.902245,0.965649
2,"{'colsample_bytree': 0.6, 'subsample': 0.8}",0.901830,0.965108
4,"{'colsample_bytree': 0.7, 'subsample': 0.6}",0.900423,0.952849
12,"{'colsample_bytree': 0.9, 'subsample': 0.6}",0.899492,0.953596


* subsample = 0.7
* colsample_bytree = 0.6
* 좀 더 미세 조정을 해 보자

In [106]:
xgb_param_grid_6m_4 = {'subsample':[i/100.0 for i in range(60,85,5)],
                        'colsample_bytree':[i/100.0 for i in range(50,75,5)]}

hr_grid_6m = GridSearchCV(estimator = \
                              XGBClassifier(
                                            n_estimator = 1000
                                            ,learning_rate = 0.1
                                            ,max_depth=4
                                            ,min_child_weight=8
                                            ,gamma=0
                                            ,random_state=random_state
                                            ,n_jobs=-1,
                                           ),
                       param_grid = xgb_param_grid_6m_4,
                       scoring = 'f1',
                       n_jobs = -1,
                       cv = 4,
                       refit = False, 
                       return_train_score = True)

hr_grid_6m.fit(X_6m, y_6m)
hr_grid_df_6m = pd.DataFrame(hr_grid_6m.cv_results_)
hr_grid_df_6m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
6,"{'colsample_bytree': 0.55, 'subsample': 0.65}",0.908396,0.953704
17,"{'colsample_bytree': 0.65, 'subsample': 0.7}",0.907441,0.956650
12,"{'colsample_bytree': 0.6, 'subsample': 0.7}",0.906031,0.959275
1,"{'colsample_bytree': 0.5, 'subsample': 0.65}",0.905820,0.953294
11,"{'colsample_bytree': 0.6, 'subsample': 0.65}",0.905693,0.955583
7,"{'colsample_bytree': 0.55, 'subsample': 0.7}",0.904427,0.959682
22,"{'colsample_bytree': 0.7, 'subsample': 0.7}",0.904197,0.961633
16,"{'colsample_bytree': 0.65, 'subsample': 0.65}",0.903295,0.956443
3,"{'colsample_bytree': 0.5, 'subsample': 0.75}",0.903024,0.961072
13,"{'colsample_bytree': 0.6, 'subsample': 0.75}",0.902730,0.961107


* subsample = 0.65
* colsample_bytree = 0.55

### 1-2-1-4. Regularization 가중치 조정

In [107]:
xgb_param_grid_6m_4 = {'reg_alpha':[1e-5, 0.01, 0.1, 1, 100]}

hr_grid_6m = GridSearchCV(estimator = \
                              XGBClassifier(
                                            n_estimator = 1000
                                            ,learning_rate = 0.1
                                            ,max_depth=4
                                            ,min_child_weight=8
                                            ,gamma=0
                                            ,colsample_bytree = 0.55
                                            ,subsample = 0.65
                                            ,random_state=random_state
                                            ,n_jobs=-1,
                                           ),
                       param_grid = xgb_param_grid_6m_4,
                       scoring = 'f1',
                       n_jobs = -1,
                       cv = 4,
                       refit = False, 
                       return_train_score = True)

hr_grid_6m.fit(X_6m, y_6m)
hr_grid_df_6m = pd.DataFrame(hr_grid_6m.cv_results_)
hr_grid_df_6m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
0,{'reg_alpha': 1e-05},0.908396,0.953704
3,{'reg_alpha': 1},0.906328,0.950171
2,{'reg_alpha': 0.1},0.906059,0.951957
1,{'reg_alpha': 0.01},0.905678,0.951914
4,{'reg_alpha': 100},0.000000,0.000000


In [109]:
xgb_param_grid_6m_5 = {'reg_alpha':[0, 1e-06, 1e-05, 1e-04, 1e-03]}

hr_grid_6m = GridSearchCV(estimator = \
                              XGBClassifier(
                                            n_estimator = 1000
                                            ,learning_rate = 0.1
                                            ,max_depth=4
                                            ,min_child_weight=8
                                            ,gamma=0
                                            ,colsample_bytree = 0.55
                                            ,subsample = 0.65
                                            ,random_state=random_state
                                            ,n_jobs=-1,
                                           ),
                       param_grid = xgb_param_grid_6m_5,
                       scoring = 'f1',
                       n_jobs = -1,
                       cv = 4,
                       refit = False,
                       return_train_score = True)

hr_grid_6m.fit(X_6m, y_6m)
hr_grid_df_6m = pd.DataFrame(hr_grid_6m.cv_results_)
hr_grid_df_6m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
0,{'reg_alpha': 0},0.908396,0.953704
1,{'reg_alpha': 1e-06},0.908396,0.953704
2,{'reg_alpha': 1e-05},0.908396,0.953704
3,{'reg_alpha': 0.0001},0.908396,0.953704
4,{'reg_alpha': 0.001},0.908396,0.953704


* reg_alpha = 0

In [110]:
xgb_param_grid_6m_6 = {'reg_lambda':[1e-5, 0.01, 0.1, 1, 100]}

hr_grid_6m = GridSearchCV(estimator = \
                              XGBClassifier(
                                            n_estimator = 1000
                                            ,learning_rate = 0.1
                                            ,max_depth=4
                                            ,min_child_weight=8
                                            ,gamma=0
                                            ,colsample_bytree = 0.55
                                            ,subsample = 0.65
                                            ,reg_alpha = 0
                                            ,random_state=random_state
                                            ,n_jobs=-1,
                                           ),
                       param_grid = xgb_param_grid_6m_6,
                       scoring = 'f1',
                       n_jobs = -1,
                       cv = 4,
                       refit = False, 
                       return_train_score = True)

hr_grid_6m.fit(X_6m, y_6m)
hr_grid_df_6m = pd.DataFrame(hr_grid_6m.cv_results_)
hr_grid_df_6m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
3,{'reg_lambda': 1},0.908396,0.953704
1,{'reg_lambda': 0.01},0.902047,0.957836
2,{'reg_lambda': 0.1},0.900429,0.957402
0,{'reg_lambda': 1e-05},0.899345,0.958262
4,{'reg_lambda': 100},0.883153,0.923436


* reg_lambda와 reg_alpha 값은 기본 값에서 달라지지 않음

### 1-2-1-5. learning rate와 n_estimator 조정

In [111]:
xgb_param_grid_6m_8 = {'n_estimator':[1000, 3000, 5000, 7000]
                      ,'learning_rate':[1e-3, 1e-2, 0.1]}

hr_grid_6m = GridSearchCV(estimator = \
                              XGBClassifier(max_depth=4
                                            ,min_child_weight=8
                                            ,gamma=0
                                            ,colsample_bytree = 0.55
                                            ,subsample = 0.65
                                            ,reg_alpha = 0
                                            ,reg_lambda = 1
                                            ,random_state=random_state
                                            ,n_jobs=-1,
                                           ),
                       param_grid = xgb_param_grid_6m_8,
                       scoring = 'f1',
                       n_jobs = -1,
                       cv = 4,
                       refit = False, 
                       return_train_score = True)

hr_grid_6m.fit(X_6m, y_6m)
hr_grid_df_6m = pd.DataFrame(hr_grid_6m.cv_results_)
hr_grid_df_6m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
8,"{'learning_rate': 0.1, 'n_estimator': 1000}",0.908396,0.953704
9,"{'learning_rate': 0.1, 'n_estimator': 3000}",0.908396,0.953704
10,"{'learning_rate': 0.1, 'n_estimator': 5000}",0.908396,0.953704
11,"{'learning_rate': 0.1, 'n_estimator': 7000}",0.908396,0.953704
4,"{'learning_rate': 0.01, 'n_estimator': 1000}",0.892917,0.919930
5,"{'learning_rate': 0.01, 'n_estimator': 3000}",0.892917,0.919930
6,"{'learning_rate': 0.01, 'n_estimator': 5000}",0.892917,0.919930
7,"{'learning_rate': 0.01, 'n_estimator': 7000}",0.892917,0.919930
0,"{'learning_rate': 0.001, 'n_estimator': 1000}",0.876618,0.912470
1,"{'learning_rate': 0.001, 'n_estimator': 3000}",0.876618,0.912470


### 1-2-1-6. 튜닝한 하이퍼파라미터를 사용한 모델 성능측정

In [112]:
best_hp_tuned_model_6m = XGBClassifier(learning_rate=0.1
                                        ,n_estimator=1000
                                        ,max_depth=4
                                        ,min_child_weight=8
                                        ,gamma=0
                                        ,colsample_bytree = 0.55
                                        ,subsample = 0.65
                                        ,reg_alpha = 0
                                        ,reg_lambda = 1
                                        ,random_state=random_state
                                        ,n_jobs=-1,
                                    )

best_hp_tuned_model_6m_nonsample = sav.suffle_and_validation(X_6m, y_6m, best_hp_tuned_model_6m, n_splits=30, random_state=random_state, test_size=0.25)

<class 'xgboost.sklearn.XGBClassifier'>
30회 최종평균결과
Train
생존 | precision : 0.994, recall : 0.991, f1-score:0.993
부도 | precision : 0.948, recall : 0.964, f1-score:0.956
accuracy : 0.987
Test
생존 | precision : 0.986, recall : 0.985, f1-score:0.985
부도 | precision : 0.912, recall : 0.916, f1-score:0.913
accuracy : 0.975


**Test 세트의 점수에는 거의 변동이 없지만, Train 세트의 과적합은 일정 부분 해소된 것을 알 수 있음**

* 하이퍼파라미터 튜닝 전 <br>
XGboost<br>
30회 최종평균결과<br>
Train<br>
생존 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
부도 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
accuracy : 1.0<br>
Test<br>
생존 | precision : 0.986, recall : 0.984, f1-score:0.985<br>
부도 | precision : 0.909, recall : 0.919, f1-score:0.913<br>
accuracy : 0.975<br>

* 하이퍼파라미터 튜닝 후 <br>
30회 최종평균결과<br>
Train<br>
생존 | precision : 0.994, recall : 0.991, f1-score:0.993<br>
부도 | precision : 0.948, recall : 0.964, f1-score:0.956<br>
accuracy : 0.987<br>
Test<br>
생존 | precision : 0.986, recall : 0.985, f1-score:0.985<br>
부도 | precision : 0.912, recall : 0.916, f1-score:0.913<br>
accuracy : 0.975<br>

In [113]:
best_hp_tuned_model_6m = XGBClassifier(learning_rate=0.1
                                        ,n_estimator=1000
                                        ,max_depth=4
                                        ,min_child_weight=8
                                        ,gamma=0
                                        ,colsample_bytree = 0.55
                                        ,subsample = 0.65
                                        ,reg_alpha = 0
                                        ,reg_lambda = 1
                                        ,random_state=random_state
                                        ,n_jobs=-1,
                                    )

best_hp_tuned_model_6m.fit(X_6m, y_6m) # 전체 데이터 학습 후 저장

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.55, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=8, missing=nan, monotone_constraints=None,
              n_estimator=1000, n_estimators=100, n_jobs=-1,
              num_parallel_tree=1, objective='binary:logistic', random_state=20,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.65,
              tree_method=None, validate_parameters=False, verbosity=None)

In [114]:
with open('best_hp_tuned_model_6m.pickle', 'wb') as f:
    pickle.dump(best_hp_tuned_model_6m, f)

## 1-2-2. feature importance 확인

* importances가 0인 항목들이 데이터 범위 중간에 위치한 데이터 (예 : 총 12개월의 유가 데이터 중 6개월 째 유가 데이터) 인 경우가 대다수여서,<br>해당 피쳐들을 제외하는 작업이 부자연스러웠으므로 피쳐 제외 없이 그대로 진행함

In [115]:
column_for_table_2 = ['stock_code'
    ,'Y-1_총자산(천원)'
    ,'Y-1_현금및현금성자산(천원)'
    ,'Y-1_총부채(천원)'
    ,'Y-1_총자본(천원)'
    ,'Y-1_매출액(천원)'
    ,'Y-1_당기순이익(천원)'
    ,'Y-1_세전계속사업이익(천원)'
    ,'Y-1_영업이익(천원)'
    ,'Y-1_매출총이익(천원)'
    ,'Y-1_차입금의존도(p)'
    ,'Y-1_당기순이익률(p)'
    ,'Y-1_세전계속사업이익률(p)'
    ,'Y-1_영업이익률(p)'
    ,'Y-1_매출총이익률(p)'
    ,'Y-1_총자산회전율(회)'
    ,'Y-1_총자산증가율(p)'
    ,'Y-1_총자산부채비율(p)'
    ,'Y-2_총자산(천원)'
    ,'Y-2_현금및현금성자산(천원)'
    ,'Y-2_총부채(천원)'
    ,'Y-2_총자본(천원)'
    ,'Y-2_매출액(천원)'
    ,'Y-2_당기순이익(천원)'
    ,'Y-2_세전계속사업이익(천원)'
    ,'Y-2_영업이익(천원)'
    ,'Y-2_매출총이익(천원)'
    ,'Y-2_차입금의존도(p)'
    ,'Y-2_당기순이익률(p)'
    ,'Y-2_세전계속사업이익률(p)'
    ,'Y-2_영업이익률(p)'
    ,'Y-2_매출총이익률(p)'
    ,'Y-2_총자산회전율(회)'
    ,'Y-2_총자산증가율(p)'
    ,'Y-2_총자산부채비율(p)'
    ,'R1-R2_외국인계매도비율변화'
    ,'R2-R3_외국인계매도비율변화'
    ,'R3-R4_외국인계매도비율변화'
    ,'R1-R2_외국인계매수비율변화'
    ,'R2-R3_외국인계매수비율변화'
    ,'R3-R4_외국인계매수비율변화'
    ,'R1-R2_기관계매도비율변화'
    ,'R2-R3_기관계매도비율변화'
    ,'R3-R4_기관계매도비율변화'
    ,'R1-R2_기관계매수비율변화'
    ,'R2-R3_기관계매수비율변화'
    ,'R3-R4_기관계매수비율변화'
    ,'수익률 (1개월)(p)'
    ,'수익률 (3개월)(p)'
    ,'수익률 (6개월)(p)'
    ,'부도기사비율_m1'
    ,'부도기사비율_m2'
    ,'부도기사비율_m3'
    ,'부도기사비율_m4'
    ,'부도기사비율_m5'
    ,'부도기사비율_m6'
    ,'부도기사비율_m7'
    ,'부도기사비율_m8'
    ,'부도기사비율_m9'
    ,'부도기사비율_m10'
    ,'부도기사비율_m11'
    ,'부도기사비율_m12'
    ,'n_opinion_in_r3'
    ,'m1_oil_price'
    ,'m2_oil_price'
    ,'m3_oil_price'
    ,'m4_oil_price'
    ,'m5_oil_price'
    ,'m6_oil_price'
    ,'m7_oil_price'
    ,'m8_oil_price'
    ,'m9_oil_price'
    ,'m10_oil_price'
    ,'m11_oil_price'
    ,'m12_oil_price'
    ,'m1_oil_price_roc_from_month_year_earlier'
    ,'m2_oil_price_roc_from_month_year_earlier'
    ,'m3_oil_price_roc_from_month_year_earlier'
    ,'m4_oil_price_roc_from_month_year_earlier'
    ,'m5_oil_price_roc_from_month_year_earlier'
    ,'m6_oil_price_roc_from_month_year_earlier'
    ,'m7_oil_price_roc_from_month_year_earlier'
    ,'m8_oil_price_roc_from_month_year_earlier'
    ,'m9_oil_price_roc_from_month_year_earlier'
    ,'m10_oil_price_roc_from_month_year_earlier'
    ,'m11_oil_price_roc_from_month_year_earlier'
    ,'m12_oil_price_roc_from_month_year_earlier'
    ,'m1_CD유통수익률(91일)(p)'
    ,'m2_CD유통수익률(91일)(p)'
    ,'m3_CD유통수익률(91일)(p)'
    ,'m4_CD유통수익률(91일)(p)'
    ,'m5_CD유통수익률(91일)(p)'
    ,'m6_CD유통수익률(91일)(p)'
    ,'m7_CD유통수익률(91일)(p)'
    ,'m8_CD유통수익률(91일)(p)'
    ,'m9_CD유통수익률(91일)(p)'
    ,'m10_CD유통수익률(91일)(p)'
    ,'m11_CD유통수익률(91일)(p)'
    ,'m12_CD유통수익률(91일)(p)'
    ,'m1_국고채(3년)(p)'
    ,'m2_국고채(3년)(p)'
    ,'m3_국고채(3년)(p)'
    ,'m4_국고채(3년)(p)'
    ,'m5_국고채(3년)(p)'
    ,'m6_국고채(3년)(p)'
    ,'m7_국고채(3년)(p)'
    ,'m8_국고채(3년)(p)'
    ,'m9_국고채(3년)(p)'
    ,'m10_국고채(3년)(p)'
    ,'m11_국고채(3년)(p)'
    ,'m12_국고채(3년)(p)'
    ,'m1_원달러환율(매매기준율)'
    ,'m2_원달러환율(매매기준율)'
    ,'m3_원달러환율(매매기준율)'
    ,'m4_원달러환율(매매기준율)'
    ,'m5_원달러환율(매매기준율)'
    ,'m6_원달러환율(매매기준율)'
    ,'m7_원달러환율(매매기준율)'
    ,'m8_원달러환율(매매기준율)'
    ,'m9_원달러환율(매매기준율)'
    ,'m10_원달러환율(매매기준율)'
    ,'m11_원달러환율(매매기준율)'
    ,'m12_원달러환율(매매기준율)'
    ,'y1_gdp_growth_rate'
    ,'y2_gdp_growth_rate'
    ,'y3_gdp_growth_rate'
    ,'bankruptcy'
    ]

feature_score_dict = {}
feature_score = best_hp_tuned_model_6m.feature_importances_
for idx, each_feature_name in enumerate(column_for_table_2[1:-1]):
    feature_score_dict[each_feature_name] = feature_score[idx]
feature_score_dict_sorted = sorted(feature_score_dict.items(), key=(lambda x:x[1]), reverse=True)
feature_score_dict_sorted

[('y3_gdp_growth_rate', 0.177536),
 ('m1_국고채(3년)(p)', 0.15438932),
 ('m1_CD유통수익률(91일)(p)', 0.10057649),
 ('m2_국고채(3년)(p)', 0.06843937),
 ('Y-1_당기순이익(천원)', 0.03417825),
 ('Y-1_총자본(천원)', 0.029550092),
 ('Y-1_세전계속사업이익(천원)', 0.029015683),
 ('Y-1_매출총이익(천원)', 0.022137957),
 ('Y-1_영업이익(천원)', 0.017542228),
 ('m3_oil_price_roc_from_month_year_earlier', 0.015072987),
 ('Y-1_당기순이익률(p)', 0.013746093),
 ('수익률 (6개월)(p)', 0.013243969),
 ('수익률 (1개월)(p)', 0.011563726),
 ('R2-R3_기관계매도비율변화', 0.011023914),
 ('Y-1_세전계속사업이익률(p)', 0.010802179),
 ('Y-1_총자산(천원)', 0.009247801),
 ('m10_oil_price_roc_from_month_year_earlier', 0.008908799),
 ('m2_oil_price_roc_from_month_year_earlier', 0.008148874),
 ('R1-R2_기관계매도비율변화', 0.007890688),
 ('m12_oil_price_roc_from_month_year_earlier', 0.007839754),
 ('수익률 (3개월)(p)', 0.007839599),
 ('m3_국고채(3년)(p)', 0.0077693867),
 ('Y-2_영업이익률(p)', 0.00725673),
 ('Y-2_세전계속사업이익률(p)', 0.0071438644),
 ('m10_국고채(3년)(p)', 0.007138497),
 ('Y-2_당기순이익률(p)', 0.0068240166),
 ('부도기사비율_m5', 0.00657

## 1-3. 3개월 예측

In [158]:
RandomForest_clf = RandomForestClassifier(random_state=random_state, n_jobs=-1)
GradientBoosting_clf = GradientBoostingClassifier(random_state=random_state)
XGBoost_clf = XGBClassifier(random_state=random_state, n_jobs=-1)

In [70]:
pre3m_fi2y = create_df_for_ml(prediction_range='3m', years=2)
X_3m = pre3m_fi2y[0]
y_3m = pre3m_fi2y[1]

create_temp_table
create_table_for_dt_based_model_stock
create_temp_earning_rate_query
create_temp_audit_report_query
final_select_query


In [159]:
rand_df_3m_nonsample = sav.suffle_and_validation(X_3m, y_3m, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25)
rand_df_3m_rand_RandomOverSampling = sav.suffle_and_validation(X_3m, y_3m, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='RandomOverSampling')
rand_df_3m_rand_ADASYN = sav.suffle_and_validation(X_3m, y_3m, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='ADASYN')
rand_df_3m_rand_SMOTE = sav.suffle_and_validation(X_3m, y_3m, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='SMOTE')
GradientBoosting_df_3m_nonsample = sav.suffle_and_validation(X_3m, y_3m, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25)
GradientBoosting_df_3m_RandomOverSampling = sav.suffle_and_validation(X_3m, y_3m, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='RandomOverSampling')
GradientBoosting_df_3m_ADASYN = sav.suffle_and_validation(X_3m, y_3m, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='ADASYN')
GradientBoosting_df_3m_SMOTE = sav.suffle_and_validation(X_3m, y_3m, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='SMOTE')
XGboost_df_3m_nonsample = sav.suffle_and_validation(X_3m, y_3m, XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25)
XGboost_df_3m_RandomOverSampling = sav.suffle_and_validation(X_3m, y_3m, XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='RandomOverSampling')
XGboost_df_3m_ADASYN = sav.suffle_and_validation(X_3m, y_3m,XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='ADASYN')
XGboost_df_3m_SMOTE = sav.suffle_and_validation(X_3m, y_3m, XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='SMOTE')

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
30회 최종평균결과
Train
생존 | precision : 1.0, recall : 1.0, f1-score:1.0
부도 | precision : 1.0, recall : 1.0, f1-score:1.0
accuracy : 1.0
Test
생존 | precision : 0.993, recall : 0.983, f1-score:0.988
부도 | precision : 0.907, recall : 0.956, f1-score:0.93
accuracy : 0.979
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
30회 최종평균결과_RandomOverSampling
Train
생존 | precision : 1.0, recall : 1.0, f1-score:1.0
부도 | precision : 1.0, recall : 1.0, f1-score:1.0
accuracy : 1.0
Test
생존 | precision : 0.994, recall : 0.983, f1-score:0.988
부도 | precision : 0.907, recall : 0.965, f1-score:0.935
accuracy : 0.98
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
30회 최종평균결과_ADASYN
Train
생존 | precision : 1.0, recall : 1.0, f1-score:1.0
부도 | precision : 1.0, recall : 1.0, f1-score:1.0
accuracy : 1.0
Test
생존 | precision : 0.995, recall : 0.977, f1-score:0.986
부도 | precision : 0.88, recall : 0.971, f1-score:0.923
accuracy : 0.976
<class 'sklearn.e

* 가장 학습결과가 좋은 조합 (Test 데이터 셋 f1-score 기준) : XGboost_df_3m_RandomOverSampling

XGboost<br>
30회 최종평균결과_RandomOverSampling<br>
Train<br>
생존 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
부도 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
accuracy : 1.0<br>
Test<br>
생존 | precision : 0.994, recall : 0.985, f1-score:0.989<br>
부도 | precision : 0.916, recall : 0.963, f1-score:0.938<br>
accuracy : 0.982<br>

### 1-3-1. Hyperparameter Tuning

In [75]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
# validation fold는 오버샘플링 되면 안 되기 때문에, imblearn.pipeline을 사용한다.
# imblearn.pipeline에서는 fit 할 때에만 sampling이 적용되며, predict 할 때에는 sampling이 적용되지 않는다.

### 1-3-1-1. max_depth, min_child_weight 조정

In [116]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000, learning_rate = 0.1, random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m = {'clf__max_depth': [4, 5, 6, 7]
                       ,'clf__min_child_weight':[1, 2, 3, 4, 5]
                    }

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
0,"{'clf__max_depth': 4, 'clf__min_child_weight': 1}",0.979134,1.000000
14,"{'clf__max_depth': 6, 'clf__min_child_weight': 5}",0.979134,0.996982
9,"{'clf__max_depth': 5, 'clf__min_child_weight': 5}",0.978740,0.996850
10,"{'clf__max_depth': 6, 'clf__min_child_weight': 1}",0.978740,1.000000
3,"{'clf__max_depth': 4, 'clf__min_child_weight': 4}",0.978346,0.997638
19,"{'clf__max_depth': 7, 'clf__min_child_weight': 5}",0.978346,0.996850
15,"{'clf__max_depth': 7, 'clf__min_child_weight': 1}",0.977953,1.000000
1,"{'clf__max_depth': 4, 'clf__min_child_weight': 2}",0.977953,0.999738
13,"{'clf__max_depth': 6, 'clf__min_child_weight': 4}",0.977953,0.998425
18,"{'clf__max_depth': 7, 'clf__min_child_weight': 4}",0.977953,0.998425


* test 세트의 점수가 같다면, train 세트의 과적합이 적은 것을 선택 : max_depth=6, min_child_weight=5

In [117]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000, learning_rate = 0.1, max_depth=6, random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m_2 = {'clf__min_child_weight':[6,8,10,12,14,16]}

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m_2,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
3,{'clf__min_child_weight': 12},0.977953,0.992651
0,{'clf__min_child_weight': 6},0.977953,0.996063
2,{'clf__min_child_weight': 10},0.977559,0.993176
1,{'clf__min_child_weight': 8},0.977165,0.994882
4,{'clf__min_child_weight': 14},0.975591,0.990814
5,{'clf__min_child_weight': 16},0.975197,0.989370


* min_child_weight = 12

### 1-3-1-2. gamma 조정

In [118]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000, learning_rate = 0.1, max_depth=6, min_child_weight=12, random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m_3 = {'clf__gamma':[i/10.0 for i in range(0,5)]}

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m_3,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
0,{'clf__gamma': 0.0},0.977953,0.992651
1,{'clf__gamma': 0.1},0.977953,0.992388
3,{'clf__gamma': 0.3},0.977165,0.992520
4,{'clf__gamma': 0.4},0.977165,0.992388
2,{'clf__gamma': 0.2},0.976772,0.992257


In [119]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000, learning_rate = 0.1, max_depth=6, min_child_weight=12, random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m_3 = {'clf__gamma':[i/100 for i in range(1,11)]}

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m_3,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
0,{'clf__gamma': 0.01},0.977953,0.992388
1,{'clf__gamma': 0.02},0.977953,0.992388
7,{'clf__gamma': 0.08},0.977953,0.992388
8,{'clf__gamma': 0.09},0.977953,0.992388
9,{'clf__gamma': 0.1},0.977953,0.992388
2,{'clf__gamma': 0.03},0.977559,0.992257
3,{'clf__gamma': 0.04},0.977559,0.992257
4,{'clf__gamma': 0.05},0.977559,0.992257
6,{'clf__gamma': 0.07},0.977165,0.992388
5,{'clf__gamma': 0.06},0.976772,0.992126


In [120]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000, learning_rate = 0.1, max_depth=6, min_child_weight=12, random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m_3 = {'clf__gamma':[i/1000 for i in range(1,11)]}

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m_3,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
0,{'clf__gamma': 0.001},0.977953,0.992651
1,{'clf__gamma': 0.002},0.977953,0.992651
2,{'clf__gamma': 0.003},0.977953,0.992651
3,{'clf__gamma': 0.004},0.977953,0.992651
4,{'clf__gamma': 0.005},0.977953,0.992651
5,{'clf__gamma': 0.006},0.977953,0.992520
6,{'clf__gamma': 0.007},0.977953,0.992520
7,{'clf__gamma': 0.008},0.977953,0.992388
8,{'clf__gamma': 0.009},0.977953,0.992388
9,{'clf__gamma': 0.01},0.977953,0.992388


* gamma = 0.008

### 1-2-1-3. subsample, colsample_bytree 조정

In [121]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000,
                              learning_rate = 0.1,
                              max_depth=6,
                              min_child_weight=12,
                              gamma=0.008,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m_4 = {'clf__subsample':[i/10.0 for i in range(6,10)],
                        'clf__colsample_bytree':[i/10.0 for i in range(6,10)]}

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m_4,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
11,"{'clf__colsample_bytree': 0.8, 'clf__subsample...",0.976378,0.991339
7,"{'clf__colsample_bytree': 0.7, 'clf__subsample...",0.975591,0.990551
2,"{'clf__colsample_bytree': 0.6, 'clf__subsample...",0.975197,0.989633
15,"{'clf__colsample_bytree': 0.9, 'clf__subsample...",0.975197,0.990551
3,"{'clf__colsample_bytree': 0.6, 'clf__subsample...",0.974803,0.990551
13,"{'clf__colsample_bytree': 0.9, 'clf__subsample...",0.974409,0.988976
6,"{'clf__colsample_bytree': 0.7, 'clf__subsample...",0.974409,0.989764
5,"{'clf__colsample_bytree': 0.7, 'clf__subsample...",0.974016,0.988189
8,"{'clf__colsample_bytree': 0.8, 'clf__subsample...",0.974016,0.986614
10,"{'clf__colsample_bytree': 0.8, 'clf__subsample...",0.974016,0.989895


In [122]:
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False).head(1)['params'].values

array([{'clf__colsample_bytree': 0.8, 'clf__subsample': 0.9}],
      dtype=object)

* subsample = 0.8
* colsample_bytree = 0.9
* 좀 더 미세 조정을 해 보자

In [123]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000,
                              learning_rate = 0.1,
                              max_depth=6,
                              min_child_weight=12,
                              gamma=0.008,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m_4 = {'clf__subsample':[i/100.0 for i in range(70,95,5)],
                        'clf__colsample_bytree':[i/100.0 for i in range(80,105,5)]}

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m_4,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
4,"{'clf__colsample_bytree': 0.8, 'clf__subsample...",0.976378,0.991339
24,"{'clf__colsample_bytree': 1.0, 'clf__subsample...",0.976378,0.990289
22,"{'clf__colsample_bytree': 1.0, 'clf__subsample...",0.976378,0.990026
19,"{'clf__colsample_bytree': 0.95, 'clf__subsampl...",0.976378,0.990945
23,"{'clf__colsample_bytree': 1.0, 'clf__subsample...",0.975984,0.990682
15,"{'clf__colsample_bytree': 0.95, 'clf__subsampl...",0.975984,0.988714
17,"{'clf__colsample_bytree': 0.95, 'clf__subsampl...",0.975197,0.990289
14,"{'clf__colsample_bytree': 0.9, 'clf__subsample...",0.975197,0.990551
8,"{'clf__colsample_bytree': 0.85, 'clf__subsampl...",0.975197,0.990157
20,"{'clf__colsample_bytree': 1.0, 'clf__subsample...",0.974803,0.988714


In [126]:
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False).head(4)['params'].values[3]

{'clf__colsample_bytree': 0.95, 'clf__subsample': 0.9}

* mean_test_score가 동일하다면, mean_train_score가 가장 작은 것을 골라서 과적합 최소화
* subsample = 0.9
* colsample_bytree = 0.95

### 1-2-1-4. Regularization 가중치 조절

In [130]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000,
                              learning_rate = 0.1,
                              max_depth=6,
                              min_child_weight=12,
                              gamma=0.008,
                              subsample = 0.9,
                              colsample_bytree = 0.95,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m_5 = {'clf__reg_alpha':[1e-5, 0.01, 0.1, 1, 100]}

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m_5,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
1,{'clf__reg_alpha': 0.01},0.976772,0.991339
0,{'clf__reg_alpha': 1e-05},0.976378,0.990945
2,{'clf__reg_alpha': 0.1},0.976378,0.991076
3,{'clf__reg_alpha': 1},0.975984,0.990289
4,{'clf__reg_alpha': 100},0.954724,0.964042


In [132]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000,
                              learning_rate = 0.1,
                              max_depth=6,
                              min_child_weight=12,
                              gamma=0.008,
                              subsample = 0.9,
                              colsample_bytree = 0.95,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m_5 = {'clf__reg_alpha':[0, 1e-04, 1e-03, 5e-03, 0.01]}

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m_5,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
4,{'clf__reg_alpha': 0.01},0.976772,0.991339
3,{'clf__reg_alpha': 0.005},0.976378,0.991339
0,{'clf__reg_alpha': 0},0.976378,0.990945
1,{'clf__reg_alpha': 0.0001},0.976378,0.991207
2,{'clf__reg_alpha': 0.001},0.976378,0.991339


* reg_alpha = 0.01

In [133]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000,
                              learning_rate = 0.1,
                              max_depth=6,
                              min_child_weight=12,
                              gamma=0.008,
                              subsample = 0.9,
                              colsample_bytree = 0.95,
                              reg_alpha = 0.01,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m_5 = {'clf__reg_lambda':[1e-5, 0.01, 0.1, 1, 100]}

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m_5,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
3,{'clf__reg_lambda': 1},0.976772,0.991339
0,{'clf__reg_lambda': 1e-05},0.975591,0.991470
1,{'clf__reg_lambda': 0.01},0.975591,0.991470
2,{'clf__reg_lambda': 0.1},0.975197,0.991339
4,{'clf__reg_lambda': 100},0.967323,0.981496


In [135]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000,
                              learning_rate = 0.1,
                              max_depth=6,
                              min_child_weight=12,
                              gamma=0.008,
                              subsample = 0.9,
                              colsample_bytree = 0.95,
                              reg_alpha = 0.01,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m_5 = {'clf__reg_lambda':[0.1, 0.25, 0.5, 0.75, 1, 25, 50, 75, 100]}

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m_5,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
4,{'clf__reg_lambda': 1},0.976772,0.991339
1,{'clf__reg_lambda': 0.25},0.976378,0.991207
3,{'clf__reg_lambda': 0.75},0.975984,0.991339
0,{'clf__reg_lambda': 0.1},0.975197,0.991339
2,{'clf__reg_lambda': 0.5},0.975197,0.991076
5,{'clf__reg_lambda': 25},0.972441,0.986089
6,{'clf__reg_lambda': 50},0.970079,0.984514
7,{'clf__reg_lambda': 75},0.968898,0.981890
8,{'clf__reg_lambda': 100},0.967323,0.981496


* reg_lambda = 1

### 1-2-1-5. learning rate와 n_estimator 조정

In [136]:
model_3m_randomOver = Pipeline([
        ('sampling', RandomOverSampler(random_state=random_state)),
        ('clf', XGBClassifier(max_depth=6,
                              min_child_weight=12,
                              gamma=0.008,
                              subsample = 0.9,
                              colsample_bytree = 0.95,
                              reg_alpha = 0.01,
                              reg_lambda = 1,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_3m_6 = {'clf__n_estimator':[1000, 3000, 5000, 7000]
                      ,'clf__learning_rate':[1e-3, 1e-2, 0.1]}

hr_grid_3m = GridSearchCV(estimator=model_3m_randomOver,
                       param_grid=xgb_param_grid_3m_6,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_3m.fit(X_3m, y_3m)
hr_grid_df_3m = pd.DataFrame(hr_grid_3m.cv_results_)
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
8,"{'clf__learning_rate': 0.1, 'clf__n_estimator'...",0.976772,0.991339
9,"{'clf__learning_rate': 0.1, 'clf__n_estimator'...",0.976772,0.991339
10,"{'clf__learning_rate': 0.1, 'clf__n_estimator'...",0.976772,0.991339
11,"{'clf__learning_rate': 0.1, 'clf__n_estimator'...",0.976772,0.991339
4,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",0.959055,0.974278
5,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",0.959055,0.974278
6,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",0.959055,0.974278
7,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",0.959055,0.974278
0,"{'clf__learning_rate': 0.001, 'clf__n_estimato...",0.957480,0.973622
1,"{'clf__learning_rate': 0.001, 'clf__n_estimato...",0.957480,0.973622


In [137]:
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False).head(1)['params'].values

array([{'clf__learning_rate': 0.1, 'clf__n_estimator': 1000}],
      dtype=object)

### 1-3-1-6. 튜닝한 하이퍼파라미터를 사용한 모델 성능측정

In [138]:
best_hp_tuned_model_3m = XGBClassifier(learning_rate=0.1,
                                       n_estimator=1000,
                                       max_depth=6,
                                       min_child_weight=12,
                                       gamma=0.008,
                                       subsample = 0.9,
                                       colsample_bytree = 0.95,
                                       reg_alpha = 0.01,
                                       reg_lambda = 1,
                                       random_state=random_state, n_jobs=-1)

best_hp_tuned_model_3m_RandomOverSampling = sav.suffle_and_validation(X_3m, y_3m, best_hp_tuned_model_3m, n_splits=30, random_state=random_state, test_size=0.25, oversampling='RandomOverSampling')

<class 'xgboost.sklearn.XGBClassifier'>
30회 최종평균결과_RandomOverSampling
Train
생존 | precision : 0.999, recall : 0.989, f1-score:0.994
부도 | precision : 0.989, recall : 0.999, f1-score:0.994
accuracy : 0.994
Test
생존 | precision : 0.996, recall : 0.979, f1-score:0.987
부도 | precision : 0.888, recall : 0.976, f1-score:0.929
accuracy : 0.978


* **하이퍼파라미터 튜닝 후 Train 세트의 점수가 소폭 내려갔으나, 과적합을 해소했다기에는 어려운 수준으로 보임**
    * Test 세트의 점수도 하강하였으나, 부도예측에서 가장 중요하다고 볼 수 있는 부도 기업 예측의 Recall이 상승하였으므로 튜닝 후 모델을 사용


* 하이퍼파라미터 튜닝 전<br>
XGboost<br>
30회 최종평균결과_RandomOverSampling<br>
Train<br>
생존 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
부도 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
accuracy : 1.0<br>
Test<br>
생존 | precision : 0.994, recall : 0.985, f1-score:0.989<br>
부도 | precision : 0.916, recall : 0.963, f1-score:0.938<br>
accuracy : 0.982<br>

* 하이퍼파라미터 튜닝 후<br>
XGboost<br>
30회 최종평균결과_RandomOverSampling<br>
Train<br>
생존 | precision : 0.999, recall : 0.989, f1-score:0.994<br>
부도 | precision : 0.989, recall : 0.999, f1-score:0.994<br>
accuracy : 0.994<br>
Test<br>
생존 | precision : 0.996, recall : 0.979, f1-score:0.987<br>
부도 | precision : 0.888, recall : 0.976, f1-score:0.929<br>
accuracy : 0.978<br>

In [140]:
X_3m_randsampleOver, y_3m_randsampleOver = RandomOverSampler(random_state=random_state).fit_resample(X_3m,y_3m)

In [141]:
X_3m_randsampleOver

,Y-1_총자산(천원),Y-1_현금및현금성자산(천원),Y-1_총부채(천원),Y-1_총자본(천원),Y-1_매출액(천원),Y-1_당기순이익(천원),Y-1_세전계속사업이익(천원),Y-1_영업이익(천원),Y-1_매출총이익(천원),Y-1_차입금의존도(p),...,m6_원달러환율(매매기준율),m7_원달러환율(매매기준율),m8_원달러환율(매매기준율),m9_원달러환율(매매기준율),m10_원달러환율(매매기준율),m11_원달러환율(매매기준율),m12_원달러환율(매매기준율),y1_gdp_growth_rate,y2_gdp_growth_rate,y3_gdp_growth_rate
0,2.887228e+09,15998000.0,2.197262e+09,689966000.0,281141000.0,30311000.0,37065000.0,42307000.0,281141000.0,14.02,...,1166.34,1166.69,1184.32,1192.95,1184.87,1166.27,1166.17,0.031,0.077,0.049
1,3.869358e+07,449801.0,4.206940e+07,-3375816.0,56129758.0,-15379698.0,-15379698.0,-8882955.0,3290716.0,90.42,...,1150.85,1166.34,1166.69,1184.32,1192.95,1184.87,1166.27,0.031,0.077,0.049
2,4.960932e+07,259521.0,4.305133e+07,6557992.0,51625830.0,469886.0,774972.0,2678560.0,4504766.0,51.50,...,1150.85,1166.34,1166.69,1184.32,1192.95,1184.87,1166.27,0.031,0.077,0.049
3,9.845974e+07,2198753.0,8.763419e+06,89696324.0,62464313.0,7345991.0,9996433.0,527409.0,8468384.0,0.00,...,1177.37,1150.85,1166.34,1166.69,1184.32,1192.95,1184.87,0.031,0.077,0.049
4,6.488546e+07,8974588.0,3.043291e+07,34452551.0,30364210.0,2680136.0,2927240.0,3379793.0,9497963.0,42.46,...,1158.65,1177.37,1150.85,1166.34,1166.69,1184.32,1192.95,0.031,0.077,0.049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4341,3.973986e+07,316256.0,1.859193e+07,21147936.0,5791314.0,-9612901.0,-8745705.0,-7561954.0,-815426.0,30.47,...,1166.45,1164.23,1175.25,1219.15,1238.40,1263.97,1261.35,0.030,0.058,0.053
4342,3.441335e+08,2842551.0,1.463909e+08,197742612.0,87695102.0,-15443063.0,-15046258.0,-13050448.0,3056754.0,18.09,...,1076.97,1087.52,1106.93,1124.78,1131.69,1143.36,1165.51,0.037,0.068,0.008
4343,7.260941e+07,1126010.0,2.885647e+07,43752943.0,39971890.0,-40440716.0,-48838203.0,-7010649.0,14301011.0,15.76,...,1263.97,1261.35,1258.71,1341.90,1461.98,1429.46,1346.10,0.030,0.058,0.053
4344,2.322778e+07,3101196.0,2.658108e+07,-3353307.0,7816517.0,-12550035.0,-12552746.0,-10487658.0,-1704639.0,80.00,...,1263.97,1261.35,1258.71,1341.90,1461.98,1429.46,1346.10,0.030,0.058,0.053


In [142]:
y_3m_randsampleOver

0       False
1        True
2        True
3       False
4        True
        ...  
4341     True
4342     True
4343     True
4344     True
4345     True
Name: bankruptcy, Length: 4346, dtype: bool

In [143]:
best_hp_tuned_model_3m = XGBClassifier(learning_rate=0.1,
                                       n_estimator=1000,
                                       max_depth=6,
                                       min_child_weight=12,
                                       gamma=0.008,
                                       subsample = 0.9,
                                       colsample_bytree = 0.95,
                                       reg_alpha = 0.01,
                                       reg_lambda = 1,
                                       random_state=random_state, n_jobs=-1)

best_hp_tuned_model_3m.fit(X_3m_randsampleOver, y_3m_randsampleOver) # 전체 데이터 학습 후 저장

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.95, gamma=0.008, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=12, missing=nan, monotone_constraints=None,
              n_estimator=1000, n_estimators=100, n_jobs=-1,
              num_parallel_tree=1, objective='binary:logistic', random_state=20,
              reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, subsample=0.9,
              tree_method=None, validate_parameters=False, verbosity=None)

In [144]:
with open('best_hp_tuned_model_3m.pickle', 'wb') as f:
    pickle.dump(best_hp_tuned_model_3m, f)

## 1-4. 1년 예측

In [160]:
RandomForest_clf = RandomForestClassifier(random_state=random_state, n_jobs=-1)
GradientBoosting_clf = GradientBoostingClassifier(random_state=random_state)
XGBoost_clf = XGBClassifier(random_state=random_state, n_jobs=-1)

In [152]:
pre1y_fi2y = create_df_for_ml(prediction_range='1y', years=2)
X_1y = pre1y_fi2y[0]
y_1y = pre1y_fi2y[1]

create_temp_table
create_table_for_dt_based_model_stock
create_temp_earning_rate_query
create_temp_audit_report_query
final_select_query


In [162]:
rand_df_1y_nonsample = sav.suffle_and_validation(X_1y, y_1y, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25)
rand_df_1y_rand_RandomOverSampling = sav.suffle_and_validation(X_1y, y_1y, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='RandomOverSampling')
rand_df_1y_rand_ADASYN = sav.suffle_and_validation(X_1y, y_1y, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='ADASYN')
rand_df_1y_rand_SMOTE = sav.suffle_and_validation(X_1y, y_1y, RandomForest_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='SMOTE')
GradientBoosting_df_1y_nonsample = sav.suffle_and_validation(X_1y, y_1y, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25)
GradientBoosting_df_1y_RandomOverSampling = sav.suffle_and_validation(X_1y, y_1y, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='RandomOverSampling')
GradientBoosting_df_1y_ADASYN = sav.suffle_and_validation(X_1y, y_1y, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='ADASYN')
GradientBoosting_df_1y_SMOTE = sav.suffle_and_validation(X_1y, y_1y, GradientBoosting_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='SMOTE')
XGboost_df_1y_nonsample = sav.suffle_and_validation(X_1y, y_1y, XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25)
XGboost_df_1y_RandomOverSampling = sav.suffle_and_validation(X_1y, y_1y, XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='RandomOverSampling')
XGboost_df_1y_ADASYN = sav.suffle_and_validation(X_1y, y_1y,XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='ADASYN')
XGboost_df_1y_SMOTE = sav.suffle_and_validation(X_1y, y_1y, XGBoost_clf, n_splits=30, random_state=random_state, test_size=0.25, oversampling='SMOTE')

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
30회 최종평균결과
Train
생존 | precision : 1.0, recall : 1.0, f1-score:1.0
부도 | precision : 1.0, recall : 1.0, f1-score:1.0
accuracy : 1.0
Test
생존 | precision : 0.988, recall : 0.976, f1-score:0.982
부도 | precision : 0.87, recall : 0.931, f1-score:0.899
accuracy : 0.969
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
30회 최종평균결과_RandomOverSampling
Train
생존 | precision : 1.0, recall : 1.0, f1-score:1.0
부도 | precision : 1.0, recall : 1.0, f1-score:1.0
accuracy : 1.0
Test
생존 | precision : 0.992, recall : 0.973, f1-score:0.982
부도 | precision : 0.859, recall : 0.952, f1-score:0.903
accuracy : 0.97
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
30회 최종평균결과_ADASYN
Train
생존 | precision : 1.0, recall : 1.0, f1-score:1.0
부도 | precision : 1.0, recall : 1.0, f1-score:1.0
accuracy : 1.0
Test
생존 | precision : 0.994, recall : 0.969, f1-score:0.981
부도 | precision : 0.843, recall : 0.964, f1-score:0.899
accuracy : 0.968
<class 'sklearn.

* 가장 학습결과가 좋은 조합 (Test 데이터 셋 f1-score 기준) : XGboost_df_1y_ADASYN

XGboost<br>
30회 최종평균결과_ADASYN<br>
Train<br>
생존 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
부도 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
accuracy : 1.0<br>
Test<br>
생존 | precision : 0.992, recall : 0.972, f1-score:0.982<br>
부도 | precision : 0.858, recall : 0.954, f1-score:0.903<br>
accuracy : 0.97<br>

### 1-4-1. Hyperparameter Tuning

In [163]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import ADASYN
# validation fold는 오버샘플링 되면 안 되기 때문에, imblearn.pipeline을 사용한다.
# imblearn.pipeline에서는 fit 할 때에만 sampling이 적용되며, predict 할 때에는 sampling이 적용되지 않는다.

### 1-4-1-1. max_depth, min_child_weight 조정

In [164]:
model_1y_ADASYN = Pipeline([
        ('sampling', ADASYN(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000, learning_rate = 0.1, random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_1y = {'clf__max_depth': [4, 5, 6, 7]
                       ,'clf__min_child_weight':[1, 2, 3, 4, 5]
                    }

hr_grid_1y = GridSearchCV(estimator=model_1y_ADASYN,
                       param_grid=xgb_param_grid_1y,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_1y.fit(X_1y, y_1y)
hr_grid_df_1y = pd.DataFrame(hr_grid_1y.cv_results_)
hr_grid_df_1y[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
12,"{'clf__max_depth': 6, 'clf__min_child_weight': 3}",0.967374,0.998912
8,"{'clf__max_depth': 5, 'clf__min_child_weight': 4}",0.966558,0.998641
6,"{'clf__max_depth': 5, 'clf__min_child_weight': 2}",0.966558,0.999456
17,"{'clf__max_depth': 7, 'clf__min_child_weight': 3}",0.966150,0.999048
16,"{'clf__max_depth': 7, 'clf__min_child_weight': 2}",0.965334,0.999592
2,"{'clf__max_depth': 4, 'clf__min_child_weight': 3}",0.964927,0.998777
7,"{'clf__max_depth': 5, 'clf__min_child_weight': 3}",0.964927,0.999048
15,"{'clf__max_depth': 7, 'clf__min_child_weight': 1}",0.964927,1.000000
13,"{'clf__max_depth': 6, 'clf__min_child_weight': 4}",0.964519,0.998505
11,"{'clf__max_depth': 6, 'clf__min_child_weight': 2}",0.964519,0.999592


* max_depth=6
* min_child_weight=3

### 1-4-1-2. gamma 조정

In [165]:
model_1y_ADASYN = Pipeline([
        ('sampling', ADASYN(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000, learning_rate = 0.1, max_depth=6, min_child_weight=3, random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_1y_2 = {'clf__gamma':[i/10.0 for i in range(0,5)]}

hr_grid_1y = GridSearchCV(estimator=model_1y_ADASYN,
                       param_grid=xgb_param_grid_1y_2,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_1y.fit(X_1y, y_1y)
hr_grid_df_1y = pd.DataFrame(hr_grid_1y.cv_results_)
hr_grid_df_1y[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
0,{'clf__gamma': 0.0},0.967374,0.998912
2,{'clf__gamma': 0.2},0.967374,0.999048
4,{'clf__gamma': 0.4},0.966966,0.998912
3,{'clf__gamma': 0.3},0.966558,0.998912
1,{'clf__gamma': 0.1},0.964519,0.999048


In [166]:
model_1y_ADASYN = Pipeline([
        ('sampling', ADASYN(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000, learning_rate = 0.1, max_depth=6, min_child_weight=3, random_state=random_state, n_jobs=-1))
    ])

xgb_param_grid_1y_2 = {'clf__gamma':[i/100 for i in range(1,21)]}

hr_grid_1y = GridSearchCV(estimator=model_1y_ADASYN,
                       param_grid=xgb_param_grid_1y_2,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_1y.fit(X_1y, y_1y)
hr_grid_df_1y = pd.DataFrame(hr_grid_1y.cv_results_)
hr_grid_df_1y[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
14,{'clf__gamma': 0.15},0.967781,0.998912
13,{'clf__gamma': 0.14},0.967374,0.998912
15,{'clf__gamma': 0.16},0.967374,0.998912
19,{'clf__gamma': 0.2},0.967374,0.999048
18,{'clf__gamma': 0.19},0.966966,0.999048
0,{'clf__gamma': 0.01},0.966966,0.999048
1,{'clf__gamma': 0.02},0.966558,0.999048
6,{'clf__gamma': 0.07},0.966150,0.998912
3,{'clf__gamma': 0.04},0.966150,0.998912
16,{'clf__gamma': 0.17},0.965742,0.998912


* gamma = 0.15

### 1-2-1-3. subsample, colsample_bytree 조정

In [167]:
model_1y_ADASYN = Pipeline([
        ('sampling', ADASYN(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000,
                              learning_rate = 0.1,
                              max_depth=6,
                              min_child_weight=3,
                              gamma=0.15,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_1y_3 = {'clf__subsample':[i/10.0 for i in range(6,10)],
                        'clf__colsample_bytree':[i/10.0 for i in range(6,10)]}

hr_grid_1y = GridSearchCV(estimator=model_1y_ADASYN,
                       param_grid=xgb_param_grid_1y_3,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_1y.fit(X_1y, y_1y)
hr_grid_df_1y = pd.DataFrame(hr_grid_1y.cv_results_)
hr_grid_df_1y[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
13,"{'clf__colsample_bytree': 0.9, 'clf__subsample...",0.968189,0.996873
4,"{'clf__colsample_bytree': 0.7, 'clf__subsample...",0.967374,0.994834
12,"{'clf__colsample_bytree': 0.9, 'clf__subsample...",0.967374,0.995514
8,"{'clf__colsample_bytree': 0.8, 'clf__subsample...",0.966966,0.995106
15,"{'clf__colsample_bytree': 0.9, 'clf__subsample...",0.966150,0.998912
0,"{'clf__colsample_bytree': 0.6, 'clf__subsample...",0.965742,0.994426
1,"{'clf__colsample_bytree': 0.6, 'clf__subsample...",0.965742,0.996330
5,"{'clf__colsample_bytree': 0.7, 'clf__subsample...",0.965742,0.997009
11,"{'clf__colsample_bytree': 0.8, 'clf__subsample...",0.965334,0.999048
14,"{'clf__colsample_bytree': 0.9, 'clf__subsample...",0.964927,0.998369


In [168]:
hr_grid_df_1y[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False).head(1)['params'].values

array([{'clf__colsample_bytree': 0.9, 'clf__subsample': 0.7}],
      dtype=object)

* subsample = 0.7
* colsample_bytree = 0.9
* 좀 더 미세 조정을 해 보자

In [171]:
model_1y_ADASYN = Pipeline([
        ('sampling', ADASYN(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000,
                              learning_rate = 0.1,
                              max_depth=6,
                              min_child_weight=3,
                              gamma=0.15,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_1y_3 = {'clf__subsample':[i/100.0 for i in range(60,85,5)],
                        'clf__colsample_bytree':[i/100.0 for i in range(80,120,5)]}

hr_grid_1y = GridSearchCV(estimator=model_1y_ADASYN,
                       param_grid=xgb_param_grid_1y_3,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_1y.fit(X_1y, y_1y)
hr_grid_df_1y = pd.DataFrame(hr_grid_1y.cv_results_)
hr_grid_df_1y[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
12,"{'clf__colsample_bytree': 0.9, 'clf__subsample...",0.968189,0.996873
18,"{'clf__colsample_bytree': 0.95, 'clf__subsampl...",0.968189,0.997961
8,"{'clf__colsample_bytree': 0.85, 'clf__subsampl...",0.967781,0.997417
22,"{'clf__colsample_bytree': 1.0, 'clf__subsample...",0.967374,0.996873
5,"{'clf__colsample_bytree': 0.85, 'clf__subsampl...",0.967374,0.995242
10,"{'clf__colsample_bytree': 0.9, 'clf__subsample...",0.967374,0.995514
13,"{'clf__colsample_bytree': 0.9, 'clf__subsample...",0.966966,0.997825
0,"{'clf__colsample_bytree': 0.8, 'clf__subsample...",0.966966,0.995106
7,"{'clf__colsample_bytree': 0.85, 'clf__subsampl...",0.966558,0.997281
17,"{'clf__colsample_bytree': 0.95, 'clf__subsampl...",0.966558,0.997009


In [175]:
hr_grid_df_1y[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False).head(1)['params'].values

array([{'clf__colsample_bytree': 0.9, 'clf__subsample': 0.7}],
      dtype=object)

* subsample = 0.7
* colsample_bytree = 0.9

### 1-2-1-4. Regularization 가중치 조절

In [176]:
model_1y_ADASYN = Pipeline([
        ('sampling', ADASYN(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000,
                              learning_rate = 0.1,
                              max_depth=6,
                              min_child_weight=3,
                              gamma=0.15,
                              subsample = 0.7,
                              colsample_bytree = 0.9,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_1y_4 = {'clf__reg_alpha':[1e-5, 0.01, 0.1, 1, 100]}

hr_grid_1y = GridSearchCV(estimator=model_1y_ADASYN,
                       param_grid=xgb_param_grid_1y_4,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_1y.fit(X_1y, y_1y)
hr_grid_df_1y = pd.DataFrame(hr_grid_1y.cv_results_)
hr_grid_df_1y[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
1,{'clf__reg_alpha': 0.01},0.968597,0.997009
0,{'clf__reg_alpha': 1e-05},0.968189,0.996873
2,{'clf__reg_alpha': 0.1},0.965742,0.997145
3,{'clf__reg_alpha': 1},0.965334,0.995106
4,{'clf__reg_alpha': 100},0.946574,0.956634


In [177]:
model_1y_ADASYN = Pipeline([
        ('sampling', ADASYN(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000,
                              learning_rate = 0.1,
                              max_depth=6,
                              min_child_weight=3,
                              gamma=0.15,
                              subsample = 0.7,
                              colsample_bytree = 0.9,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_1y_4 = {'clf__reg_alpha':[1e-4 ,1e-3, 5e-3, 0.01, 0.03, 0.05]}

hr_grid_1y = GridSearchCV(estimator=model_1y_ADASYN,
                       param_grid=xgb_param_grid_1y_4,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_1y.fit(X_1y, y_1y)
hr_grid_df_1y = pd.DataFrame(hr_grid_1y.cv_results_)
hr_grid_df_1y[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
3,{'clf__reg_alpha': 0.01},0.968597,0.997009
4,{'clf__reg_alpha': 0.03},0.968597,0.997145
0,{'clf__reg_alpha': 0.0001},0.968189,0.996873
2,{'clf__reg_alpha': 0.005},0.968189,0.997009
1,{'clf__reg_alpha': 0.001},0.967781,0.997281
5,{'clf__reg_alpha': 0.05},0.966558,0.996873


* reg_alpha = 0.01

In [179]:
model_1y_ADASYN = Pipeline([
        ('sampling', ADASYN(random_state=random_state)),
        ('clf', XGBClassifier(n_estimator = 1000,
                              learning_rate = 0.1,
                              max_depth=6,
                              min_child_weight=3,
                              gamma=0.15,
                              subsample = 0.7,
                              colsample_bytree = 0.9,
                              reg_alpha = 0.01,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_1y_4 = {'clf__reg_lambda':[0, 1e-5, 0.01, 0.1, 1, 100]}

hr_grid_1y = GridSearchCV(estimator=model_1y_ADASYN,
                       param_grid=xgb_param_grid_1y_4,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_1y.fit(X_1y, y_1y)
hr_grid_df_1y = pd.DataFrame(hr_grid_1y.cv_results_)
hr_grid_df_1y[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
0,{'clf__reg_lambda': 0},0.970228,0.997961
1,{'clf__reg_lambda': 1e-05},0.970228,0.997961
3,{'clf__reg_lambda': 0.1},0.969413,0.997961
2,{'clf__reg_lambda': 0.01},0.969005,0.998097
4,{'clf__reg_lambda': 1},0.968597,0.997009
5,{'clf__reg_lambda': 100},0.955139,0.975530


* reg_lambda = 0

### 1-2-1-5. learning rate와 n_estimator 조정

In [180]:
model_1y_ADASYN = Pipeline([
        ('sampling', ADASYN(random_state=random_state)),
        ('clf', XGBClassifier(
                              max_depth=6,
                              min_child_weight=3,
                              gamma=0.15,
                              subsample = 0.7,
                              colsample_bytree = 0.9,
                              reg_alpha = 0.01,
                              reg_lambda = 0,
                              random_state=random_state, n_jobs=-1))
    ])


xgb_param_grid_1y_5 = {'clf__n_estimator':[1000, 3000, 5000, 7000]
                      ,'clf__learning_rate':[1e-3, 1e-2, 0.1]}

hr_grid_1y = GridSearchCV(estimator=model_1y_ADASYN,
                       param_grid=xgb_param_grid_1y_5,
                       scoring='f1_micro',
                       n_jobs=8,
                       cv=4,
                       refit=True, 
                       return_train_score=True)

hr_grid_1y.fit(X_1y, y_1y)
hr_grid_df_1y = pd.DataFrame(hr_grid_1y.cv_results_)
hr_grid_df_1y[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False)

,params,mean_test_score,mean_train_score
8,"{'clf__learning_rate': 0.1, 'clf__n_estimator'...",0.970228,0.997961
9,"{'clf__learning_rate': 0.1, 'clf__n_estimator'...",0.970228,0.997961
10,"{'clf__learning_rate': 0.1, 'clf__n_estimator'...",0.970228,0.997961
11,"{'clf__learning_rate': 0.1, 'clf__n_estimator'...",0.970228,0.997961
4,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",0.960440,0.976482
5,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",0.960440,0.976482
6,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",0.960440,0.976482
7,"{'clf__learning_rate': 0.01, 'clf__n_estimator...",0.960440,0.976482
0,"{'clf__learning_rate': 0.001, 'clf__n_estimato...",0.956362,0.972675
1,"{'clf__learning_rate': 0.001, 'clf__n_estimato...",0.956362,0.972675


In [181]:
hr_grid_df_3m[['params', 'mean_test_score', 'mean_train_score']].sort_values(by=['mean_test_score'], ascending=False).head(1)['params'].values

array([{'clf__learning_rate': 0.1, 'clf__n_estimator': 1000}],
      dtype=object)

### 1-3-1-6. 튜닝한 하이퍼파라미터를 사용한 모델 성능측정

In [182]:
best_hp_tuned_model_1y = XGBClassifier(learning_rate=0.1,
                                       n_estimator=1000,
                                       max_depth=6,
                                       min_child_weight=3,
                                       gamma=0.15,
                                       subsample = 0.7,
                                       colsample_bytree = 0.9,
                                       reg_alpha = 0.01,
                                       reg_lambda = 0,
                                       random_state=random_state, n_jobs=-1)

best_hp_tuned_model_1y_ADASYN = sav.suffle_and_validation(X_1y, y_1y, best_hp_tuned_model_1y, n_splits=30, random_state=random_state, test_size=0.25, oversampling='ADASYN')

<class 'xgboost.sklearn.XGBClassifier'>
30회 최종평균결과_ADASYN
Train
생존 | precision : 1.0, recall : 0.998, f1-score:0.999
부도 | precision : 0.998, recall : 1.0, f1-score:0.999
accuracy : 0.999
Test
생존 | precision : 0.992, recall : 0.971, f1-score:0.982
부도 | precision : 0.852, recall : 0.957, f1-score:0.901
accuracy : 0.969


* **하이퍼파라미터 튜닝 후 Train 세트는 30회 검증이 전부 1.0이 아니라는 것에 만족해야 하는 수준**
    * Test 세트의 f1-score도 하강하였으나 0.002 차이이고, 부도예측에서 가장 중요하다고 볼 수 있는 부도 기업 예측의 Recall이 상승하였으므로 튜닝 후 모델을 사용


* 하이퍼파라미터 튜닝 전<br>
XGboost<br>
30회 최종평균결과_ADASYN<br>
Train<br>
생존 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
부도 | precision : 1.0, recall : 1.0, f1-score:1.0<br>
accuracy : 1.0<br>
Test<br>
생존 | precision : 0.992, recall : 0.972, f1-score:0.982<br>
부도 | precision : 0.858, recall : 0.954, f1-score:0.903<br>
accuracy : 0.97<br>

* 하이퍼파라미터 튜닝 후<br>
XGboost<br>
30회 최종평균결과_ADASYN<br>
Train<br>
생존 | precision : 1.0, recall : 0.998, f1-score:0.999<br>
부도 | precision : 0.998, recall : 1.0, f1-score:0.999<br>
accuracy : 0.999<br>
Test<br>
생존 | precision : 0.992, recall : 0.971, f1-score:0.982<br>
부도 | precision : 0.852, recall : 0.957, f1-score:0.901<br>
accuracy : 0.969<br>

In [184]:
X_1y_ADASYN, y_1y_ADASYN = ADASYN(random_state=random_state).fit_resample(X_1y,y_1y)

In [185]:
X_1y_ADASYN

,Y-1_총자산(천원),Y-1_현금및현금성자산(천원),Y-1_총부채(천원),Y-1_총자본(천원),Y-1_매출액(천원),Y-1_당기순이익(천원),Y-1_세전계속사업이익(천원),Y-1_영업이익(천원),Y-1_매출총이익(천원),Y-1_차입금의존도(p),...,m6_원달러환율(매매기준율),m7_원달러환율(매매기준율),m8_원달러환율(매매기준율),m9_원달러환율(매매기준율),m10_원달러환율(매매기준율),m11_원달러환율(매매기준율),m12_원달러환율(매매기준율),y1_gdp_growth_rate,y2_gdp_growth_rate,y3_gdp_growth_rate
0,4.787087e+07,1.829380e+05,2.151610e+07,2.635476e+07,3.917125e+07,-8.648323e+06,-8.648323e+06,-1.592866e+06,5.001758e+06,34.210000,...,1194.020000,1199.780000,1232.820000,1232.400000,1191.190000,1179.270000,1208.910000,0.031000,0.077000,0.049000
1,4.304991e+07,7.088000e+03,2.895183e+07,1.409808e+07,3.923009e+07,-1.568804e+07,-1.536986e+07,-1.122842e+07,2.147440e+05,44.050000,...,1181.590000,1194.020000,1199.780000,1232.820000,1232.400000,1191.190000,1179.270000,0.031000,0.077000,0.049000
2,1.926910e+07,5.711510e+05,1.232127e+07,6.947833e+06,3.465550e+07,5.905550e+05,5.287550e+05,1.085345e+06,3.156246e+06,50.790000,...,1181.590000,1194.020000,1199.780000,1232.820000,1232.400000,1191.190000,1179.270000,0.031000,0.077000,0.049000
3,1.757587e+07,5.564400e+04,6.061991e+06,1.151388e+07,5.397651e+06,-1.221854e+07,-1.184585e+07,-3.558459e+06,-4.502500e+04,23.900000,...,1166.170000,1178.420000,1181.590000,1194.020000,1199.780000,1232.820000,1232.400000,0.031000,0.077000,0.049000
4,9.380172e+06,4.250560e+05,1.246787e+07,-3.087702e+06,2.353039e+07,-2.157390e+07,-2.157390e+07,-1.145796e+07,-6.778402e+06,91.050000,...,1166.170000,1178.420000,1181.590000,1194.020000,1199.780000,1232.820000,1232.400000,0.031000,0.077000,0.049000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,5.901219e+07,3.363998e+05,2.529964e+07,3.371256e+07,1.438367e+07,-1.551947e+07,-1.569905e+07,-7.996557e+06,-9.475094e+05,21.819708,...,1240.022003,1194.334652,1201.609316,1211.039413,1187.742212,1094.556121,1066.433771,0.031125,0.041682,0.038933
4188,5.443303e+07,3.556287e+05,2.383554e+07,3.059749e+07,1.843774e+07,-1.257240e+07,-1.266171e+07,-4.984642e+06,1.299263e+06,24.911385,...,1335.250190,1270.625255,1288.187595,1301.045975,1257.705201,1112.574405,1053.919974,0.030560,0.049885,0.046004
4189,6.384965e+07,2.635685e+06,3.611198e+07,2.773766e+07,2.103517e+07,-1.156180e+07,-1.065792e+07,-4.261154e+06,3.881688e+06,35.805036,...,1112.274289,1073.960430,1061.527624,1078.967444,1082.567919,1081.896133,1113.448430,0.035773,0.058426,0.012910
4190,6.473081e+07,2.037052e+06,1.471464e+07,5.001618e+07,1.100313e+07,-1.501782e+07,-1.619452e+07,-5.728366e+06,-5.243333e+05,7.836105,...,1143.808745,1138.834682,1120.114536,1148.252126,1170.023691,1155.305538,1142.539538,0.029374,0.030969,0.030626


In [186]:
y_1y_ADASYN

0        True
1       False
2       False
3        True
4        True
        ...  
4187     True
4188     True
4189     True
4190     True
4191     True
Name: bankruptcy, Length: 4192, dtype: bool

In [188]:
sum(y_1y_ADASYN) / len(y_1y_ADASYN)

0.5014312977099237

In [190]:
best_hp_tuned_model_1y = XGBClassifier(learning_rate=0.1,
                                       n_estimator=1000,
                                       max_depth=6,
                                       min_child_weight=3,
                                       gamma=0.15,
                                       subsample = 0.7,
                                       colsample_bytree = 0.9,
                                       reg_alpha = 0.01,
                                       reg_lambda = 0,
                                       random_state=random_state, n_jobs=-1)

best_hp_tuned_model_1y.fit(X_1y_ADASYN, y_1y_ADASYN) # 전체 데이터 학습 후 저장

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0.15, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimator=1000, n_estimators=100, n_jobs=-1,
              num_parallel_tree=1, objective='binary:logistic', random_state=20,
              reg_alpha=0.01, reg_lambda=0, scale_pos_weight=1, subsample=0.7,
              tree_method=None, validate_parameters=False, verbosity=None)

In [191]:
with open('best_hp_tuned_model_1y.pickle', 'wb') as f:
    pickle.dump(best_hp_tuned_model_1y, f)

# 2. 기업 하나의 최신 데이터 추출 및 평가 테스트

In [146]:
column_for_table_onlyX_3 = ['stock_code'
,'Y-1_총자산(천원)'
,'Y-1_현금및현금성자산(천원)'
,'Y-1_총부채(천원)'
,'Y-1_총자본(천원)'
,'Y-1_매출액(천원)'
,'Y-1_당기순이익(천원)'
,'Y-1_세전계속사업이익(천원)'
,'Y-1_영업이익(천원)'
,'Y-1_매출총이익(천원)'
,'Y-1_차입금의존도(p)'
,'Y-1_당기순이익률(p)'
,'Y-1_세전계속사업이익률(p)'
,'Y-1_영업이익률(p)'
,'Y-1_매출총이익률(p)'
,'Y-1_총자산회전율(회)'
,'Y-1_총자산증가율(p)'
,'Y-1_총자산부채비율(p)'
,'Y-2_총자산(천원)'
,'Y-2_현금및현금성자산(천원)'
,'Y-2_총부채(천원)'
,'Y-2_총자본(천원)'
,'Y-2_매출액(천원)'
,'Y-2_당기순이익(천원)'
,'Y-2_세전계속사업이익(천원)'
,'Y-2_영업이익(천원)'
,'Y-2_매출총이익(천원)'
,'Y-2_차입금의존도(p)'
,'Y-2_당기순이익률(p)'
,'Y-2_세전계속사업이익률(p)'
,'Y-2_영업이익률(p)'
,'Y-2_매출총이익률(p)'
,'Y-2_총자산회전율(회)'
,'Y-2_총자산증가율(p)'
,'Y-2_총자산부채비율(p)'
,'Y-3_총자산(천원)'
,'Y-3_현금및현금성자산(천원)'
,'Y-3_총부채(천원)'
,'Y-3_총자본(천원)'
,'Y-3_매출액(천원)'
,'Y-3_당기순이익(천원)'
,'Y-3_세전계속사업이익(천원)'
,'Y-3_영업이익(천원)'
,'Y-3_매출총이익(천원)'
,'Y-3_차입금의존도(p)'
,'Y-3_당기순이익률(p)'
,'Y-3_세전계속사업이익률(p)'
,'Y-3_영업이익률(p)'
,'Y-3_매출총이익률(p)'
,'Y-3_총자산회전율(회)'
,'Y-3_총자산증가율(p)'
,'Y-3_총자산부채비율(p)'
,'R1-R2_외국인계매도비율변화'
,'R2-R3_외국인계매도비율변화'
,'R3-R4_외국인계매도비율변화'
,'R1-R2_외국인계매수비율변화'
,'R2-R3_외국인계매수비율변화'
,'R3-R4_외국인계매수비율변화'
,'R1-R2_기관계매도비율변화'
,'R2-R3_기관계매도비율변화'
,'R3-R4_기관계매도비율변화'
,'R1-R2_기관계매수비율변화'
,'R2-R3_기관계매수비율변화'
,'R3-R4_기관계매수비율변화'
,'수익률 (1개월)(p)'
,'수익률 (3개월)(p)'
,'수익률 (6개월)(p)'
,'부도기사비율_m1'
,'부도기사비율_m2'
,'부도기사비율_m3'
,'부도기사비율_m4'
,'부도기사비율_m5'
,'부도기사비율_m6'
,'부도기사비율_m7'
,'부도기사비율_m8'
,'부도기사비율_m9'
,'부도기사비율_m10'
,'부도기사비율_m11'
,'부도기사비율_m12'
,'n_opinion_in_r3'
,'m1_oil_price'
,'m2_oil_price'
,'m3_oil_price'
,'m4_oil_price'
,'m5_oil_price'
,'m6_oil_price'
,'m7_oil_price'
,'m8_oil_price'
,'m9_oil_price'
,'m10_oil_price'
,'m11_oil_price'
,'m12_oil_price'
,'m1_oil_price_roc_from_month_year_earlier'
,'m2_oil_price_roc_from_month_year_earlier'
,'m3_oil_price_roc_from_month_year_earlier'
,'m4_oil_price_roc_from_month_year_earlier'
,'m5_oil_price_roc_from_month_year_earlier'
,'m6_oil_price_roc_from_month_year_earlier'
,'m7_oil_price_roc_from_month_year_earlier'
,'m8_oil_price_roc_from_month_year_earlier'
,'m9_oil_price_roc_from_month_year_earlier'
,'m10_oil_price_roc_from_month_year_earlier'
,'m11_oil_price_roc_from_month_year_earlier'
,'m12_oil_price_roc_from_month_year_earlier'
,'m1_CD유통수익률(91일)(p)'
,'m2_CD유통수익률(91일)(p)'
,'m3_CD유통수익률(91일)(p)'
,'m4_CD유통수익률(91일)(p)'
,'m5_CD유통수익률(91일)(p)'
,'m6_CD유통수익률(91일)(p)'
,'m7_CD유통수익률(91일)(p)'
,'m8_CD유통수익률(91일)(p)'
,'m9_CD유통수익률(91일)(p)'
,'m10_CD유통수익률(91일)(p)'
,'m11_CD유통수익률(91일)(p)'
,'m12_CD유통수익률(91일)(p)'
,'m1_국고채(3년)(p)'
,'m2_국고채(3년)(p)'
,'m3_국고채(3년)(p)'
,'m4_국고채(3년)(p)'
,'m5_국고채(3년)(p)'
,'m6_국고채(3년)(p)'
,'m7_국고채(3년)(p)'
,'m8_국고채(3년)(p)'
,'m9_국고채(3년)(p)'
,'m10_국고채(3년)(p)'
,'m11_국고채(3년)(p)'
,'m12_국고채(3년)(p)'
,'m1_원달러환율(매매기준율)'
,'m2_원달러환율(매매기준율)'
,'m3_원달러환율(매매기준율)'
,'m4_원달러환율(매매기준율)'
,'m5_원달러환율(매매기준율)'
,'m6_원달러환율(매매기준율)'
,'m7_원달러환율(매매기준율)'
,'m8_원달러환율(매매기준율)'
,'m9_원달러환율(매매기준율)'
,'m10_원달러환율(매매기준율)'
,'m11_원달러환율(매매기준율)'
,'m12_원달러환율(매매기준율)'
,'y1_gdp_growth_rate'
,'y2_gdp_growth_rate'
,'y3_gdp_growth_rate'
]

column_for_table_onlyX_2 = ['stock_code'
,'Y-1_총자산(천원)'
,'Y-1_현금및현금성자산(천원)'
,'Y-1_총부채(천원)'
,'Y-1_총자본(천원)'
,'Y-1_매출액(천원)'
,'Y-1_당기순이익(천원)'
,'Y-1_세전계속사업이익(천원)'
,'Y-1_영업이익(천원)'
,'Y-1_매출총이익(천원)'
,'Y-1_차입금의존도(p)'
,'Y-1_당기순이익률(p)'
,'Y-1_세전계속사업이익률(p)'
,'Y-1_영업이익률(p)'
,'Y-1_매출총이익률(p)'
,'Y-1_총자산회전율(회)'
,'Y-1_총자산증가율(p)'
,'Y-1_총자산부채비율(p)'
,'Y-2_총자산(천원)'
,'Y-2_현금및현금성자산(천원)'
,'Y-2_총부채(천원)'
,'Y-2_총자본(천원)'
,'Y-2_매출액(천원)'
,'Y-2_당기순이익(천원)'
,'Y-2_세전계속사업이익(천원)'
,'Y-2_영업이익(천원)'
,'Y-2_매출총이익(천원)'
,'Y-2_차입금의존도(p)'
,'Y-2_당기순이익률(p)'
,'Y-2_세전계속사업이익률(p)'
,'Y-2_영업이익률(p)'
,'Y-2_매출총이익률(p)'
,'Y-2_총자산회전율(회)'
,'Y-2_총자산증가율(p)'
,'Y-2_총자산부채비율(p)'
,'R1-R2_외국인계매도비율변화'
,'R2-R3_외국인계매도비율변화'
,'R3-R4_외국인계매도비율변화'
,'R1-R2_외국인계매수비율변화'
,'R2-R3_외국인계매수비율변화'
,'R3-R4_외국인계매수비율변화'
,'R1-R2_기관계매도비율변화'
,'R2-R3_기관계매도비율변화'
,'R3-R4_기관계매도비율변화'
,'R1-R2_기관계매수비율변화'
,'R2-R3_기관계매수비율변화'
,'R3-R4_기관계매수비율변화'
,'수익률 (1개월)(p)'
,'수익률 (3개월)(p)'
,'수익률 (6개월)(p)'
,'부도기사비율_m1'
,'부도기사비율_m2'
,'부도기사비율_m3'
,'부도기사비율_m4'
,'부도기사비율_m5'
,'부도기사비율_m6'
,'부도기사비율_m7'
,'부도기사비율_m8'
,'부도기사비율_m9'
,'부도기사비율_m10'
,'부도기사비율_m11'
,'부도기사비율_m12'
,'n_opinion_in_r3'
,'m1_oil_price'
,'m2_oil_price'
,'m3_oil_price'
,'m4_oil_price'
,'m5_oil_price'
,'m6_oil_price'
,'m7_oil_price'
,'m8_oil_price'
,'m9_oil_price'
,'m10_oil_price'
,'m11_oil_price'
,'m12_oil_price'
,'m1_oil_price_roc_from_month_year_earlier'
,'m2_oil_price_roc_from_month_year_earlier'
,'m3_oil_price_roc_from_month_year_earlier'
,'m4_oil_price_roc_from_month_year_earlier'
,'m5_oil_price_roc_from_month_year_earlier'
,'m6_oil_price_roc_from_month_year_earlier'
,'m7_oil_price_roc_from_month_year_earlier'
,'m8_oil_price_roc_from_month_year_earlier'
,'m9_oil_price_roc_from_month_year_earlier'
,'m10_oil_price_roc_from_month_year_earlier'
,'m11_oil_price_roc_from_month_year_earlier'
,'m12_oil_price_roc_from_month_year_earlier'
,'m1_CD유통수익률(91일)(p)'
,'m2_CD유통수익률(91일)(p)'
,'m3_CD유통수익률(91일)(p)'
,'m4_CD유통수익률(91일)(p)'
,'m5_CD유통수익률(91일)(p)'
,'m6_CD유통수익률(91일)(p)'
,'m7_CD유통수익률(91일)(p)'
,'m8_CD유통수익률(91일)(p)'
,'m9_CD유통수익률(91일)(p)'
,'m10_CD유통수익률(91일)(p)'
,'m11_CD유통수익률(91일)(p)'
,'m12_CD유통수익률(91일)(p)'
,'m1_국고채(3년)(p)'
,'m2_국고채(3년)(p)'
,'m3_국고채(3년)(p)'
,'m4_국고채(3년)(p)'
,'m5_국고채(3년)(p)'
,'m6_국고채(3년)(p)'
,'m7_국고채(3년)(p)'
,'m8_국고채(3년)(p)'
,'m9_국고채(3년)(p)'
,'m10_국고채(3년)(p)'
,'m11_국고채(3년)(p)'
,'m12_국고채(3년)(p)'
,'m1_원달러환율(매매기준율)'
,'m2_원달러환율(매매기준율)'
,'m3_원달러환율(매매기준율)'
,'m4_원달러환율(매매기준율)'
,'m5_원달러환율(매매기준율)'
,'m6_원달러환율(매매기준율)'
,'m7_원달러환율(매매기준율)'
,'m8_원달러환율(매매기준율)'
,'m9_원달러환율(매매기준율)'
,'m10_원달러환율(매매기준율)'
,'m11_원달러환율(매매기준율)'
,'m12_원달러환율(매매기준율)'
,'y1_gdp_growth_rate'
,'y2_gdp_growth_rate'
,'y3_gdp_growth_rate'
]

In [147]:
dict_for_insert_into_dataframe = {}
for each_key in column_for_table_onlyX_2:
    dict_for_insert_into_dataframe[each_key] = ''

In [148]:
dict_for_insert_into_dataframe

{'stock_code': '',
 'Y-1_총자산(천원)': '',
 'Y-1_현금및현금성자산(천원)': '',
 'Y-1_총부채(천원)': '',
 'Y-1_총자본(천원)': '',
 'Y-1_매출액(천원)': '',
 'Y-1_당기순이익(천원)': '',
 'Y-1_세전계속사업이익(천원)': '',
 'Y-1_영업이익(천원)': '',
 'Y-1_매출총이익(천원)': '',
 'Y-1_차입금의존도(p)': '',
 'Y-1_당기순이익률(p)': '',
 'Y-1_세전계속사업이익률(p)': '',
 'Y-1_영업이익률(p)': '',
 'Y-1_매출총이익률(p)': '',
 'Y-1_총자산회전율(회)': '',
 'Y-1_총자산증가율(p)': '',
 'Y-1_총자산부채비율(p)': '',
 'Y-2_총자산(천원)': '',
 'Y-2_현금및현금성자산(천원)': '',
 'Y-2_총부채(천원)': '',
 'Y-2_총자본(천원)': '',
 'Y-2_매출액(천원)': '',
 'Y-2_당기순이익(천원)': '',
 'Y-2_세전계속사업이익(천원)': '',
 'Y-2_영업이익(천원)': '',
 'Y-2_매출총이익(천원)': '',
 'Y-2_차입금의존도(p)': '',
 'Y-2_당기순이익률(p)': '',
 'Y-2_세전계속사업이익률(p)': '',
 'Y-2_영업이익률(p)': '',
 'Y-2_매출총이익률(p)': '',
 'Y-2_총자산회전율(회)': '',
 'Y-2_총자산증가율(p)': '',
 'Y-2_총자산부채비율(p)': '',
 'R1-R2_외국인계매도비율변화': '',
 'R2-R3_외국인계매도비율변화': '',
 'R3-R4_외국인계매도비율변화': '',
 'R1-R2_외국인계매수비율변화': '',
 'R2-R3_외국인계매수비율변화': '',
 'R3-R4_외국인계매수비율변화': '',
 'R1-R2_기관계매도비율변화': '',
 'R2-R3_기관계매도비율변화': '',
 'R3-R4_기관계매도비율변화': '',
 'R1-R2_기관계

In [192]:
def model_prediction(company_name, year=2):
    column_for_table_onlyX_2 = [
        'stock_code'
        ,'Y-1_총자산(천원)'
        ,'Y-1_현금및현금성자산(천원)'
        ,'Y-1_총부채(천원)'
        ,'Y-1_총자본(천원)'
        ,'Y-1_매출액(천원)'
        ,'Y-1_당기순이익(천원)'
        ,'Y-1_세전계속사업이익(천원)'
        ,'Y-1_영업이익(천원)'
        ,'Y-1_매출총이익(천원)'
        ,'Y-1_차입금의존도(p)'
        ,'Y-1_당기순이익률(p)'
        ,'Y-1_세전계속사업이익률(p)'
        ,'Y-1_영업이익률(p)'
        ,'Y-1_매출총이익률(p)'
        ,'Y-1_총자산회전율(회)'
        ,'Y-1_총자산증가율(p)'
        ,'Y-1_총자산부채비율(p)'
        ,'Y-2_총자산(천원)'
        ,'Y-2_현금및현금성자산(천원)'
        ,'Y-2_총부채(천원)'
        ,'Y-2_총자본(천원)'
        ,'Y-2_매출액(천원)'
        ,'Y-2_당기순이익(천원)'
        ,'Y-2_세전계속사업이익(천원)'
        ,'Y-2_영업이익(천원)'
        ,'Y-2_매출총이익(천원)'
        ,'Y-2_차입금의존도(p)'
        ,'Y-2_당기순이익률(p)'
        ,'Y-2_세전계속사업이익률(p)'
        ,'Y-2_영업이익률(p)'
        ,'Y-2_매출총이익률(p)'
        ,'Y-2_총자산회전율(회)'
        ,'Y-2_총자산증가율(p)'
        ,'Y-2_총자산부채비율(p)'
        ,'R1-R2_외국인계매도비율변화'
        ,'R2-R3_외국인계매도비율변화'
        ,'R3-R4_외국인계매도비율변화'
        ,'R1-R2_외국인계매수비율변화'
        ,'R2-R3_외국인계매수비율변화'
        ,'R3-R4_외국인계매수비율변화'
        ,'R1-R2_기관계매도비율변화'
        ,'R2-R3_기관계매도비율변화'
        ,'R3-R4_기관계매도비율변화'
        ,'R1-R2_기관계매수비율변화'
        ,'R2-R3_기관계매수비율변화'
        ,'R3-R4_기관계매수비율변화'
        ,'수익률 (1개월)(p)'
        ,'수익률 (3개월)(p)'
        ,'수익률 (6개월)(p)'
        ,'부도기사비율_m1'
        ,'부도기사비율_m2'
        ,'부도기사비율_m3'
        ,'부도기사비율_m4'
        ,'부도기사비율_m5'
        ,'부도기사비율_m6'
        ,'부도기사비율_m7'
        ,'부도기사비율_m8'
        ,'부도기사비율_m9'
        ,'부도기사비율_m10'
        ,'부도기사비율_m11'
        ,'부도기사비율_m12'
        ,'n_opinion_in_r3'
        ,'m1_oil_price'
        ,'m2_oil_price'
        ,'m3_oil_price'
        ,'m4_oil_price'
        ,'m5_oil_price'
        ,'m6_oil_price'
        ,'m7_oil_price'
        ,'m8_oil_price'
        ,'m9_oil_price'
        ,'m10_oil_price'
        ,'m11_oil_price'
        ,'m12_oil_price'
        ,'m1_oil_price_roc_from_month_year_earlier'
        ,'m2_oil_price_roc_from_month_year_earlier'
        ,'m3_oil_price_roc_from_month_year_earlier'
        ,'m4_oil_price_roc_from_month_year_earlier'
        ,'m5_oil_price_roc_from_month_year_earlier'
        ,'m6_oil_price_roc_from_month_year_earlier'
        ,'m7_oil_price_roc_from_month_year_earlier'
        ,'m8_oil_price_roc_from_month_year_earlier'
        ,'m9_oil_price_roc_from_month_year_earlier'
        ,'m10_oil_price_roc_from_month_year_earlier'
        ,'m11_oil_price_roc_from_month_year_earlier'
        ,'m12_oil_price_roc_from_month_year_earlier'
        ,'m1_CD유통수익률(91일)(p)'
        ,'m2_CD유통수익률(91일)(p)'
        ,'m3_CD유통수익률(91일)(p)'
        ,'m4_CD유통수익률(91일)(p)'
        ,'m5_CD유통수익률(91일)(p)'
        ,'m6_CD유통수익률(91일)(p)'
        ,'m7_CD유통수익률(91일)(p)'
        ,'m8_CD유통수익률(91일)(p)'
        ,'m9_CD유통수익률(91일)(p)'
        ,'m10_CD유통수익률(91일)(p)'
        ,'m11_CD유통수익률(91일)(p)'
        ,'m12_CD유통수익률(91일)(p)'
        ,'m1_국고채(3년)(p)'
        ,'m2_국고채(3년)(p)'
        ,'m3_국고채(3년)(p)'
        ,'m4_국고채(3년)(p)'
        ,'m5_국고채(3년)(p)'
        ,'m6_국고채(3년)(p)'
        ,'m7_국고채(3년)(p)'
        ,'m8_국고채(3년)(p)'
        ,'m9_국고채(3년)(p)'
        ,'m10_국고채(3년)(p)'
        ,'m11_국고채(3년)(p)'
        ,'m12_국고채(3년)(p)'
        ,'m1_원달러환율(매매기준율)'
        ,'m2_원달러환율(매매기준율)'
        ,'m3_원달러환율(매매기준율)'
        ,'m4_원달러환율(매매기준율)'
        ,'m5_원달러환율(매매기준율)'
        ,'m6_원달러환율(매매기준율)'
        ,'m7_원달러환율(매매기준율)'
        ,'m8_원달러환율(매매기준율)'
        ,'m9_원달러환율(매매기준율)'
        ,'m10_원달러환율(매매기준율)'
        ,'m11_원달러환율(매매기준율)'
        ,'m12_원달러환율(매매기준율)'
        ,'y1_gdp_growth_rate'
        ,'y2_gdp_growth_rate'
        ,'y3_gdp_growth_rate'
    ]

    column_for_table_onlyX_3 = [
        'stock_code'
        ,'Y-1_총자산(천원)'
        ,'Y-1_현금및현금성자산(천원)'
        ,'Y-1_총부채(천원)'
        ,'Y-1_총자본(천원)'
        ,'Y-1_매출액(천원)'
        ,'Y-1_당기순이익(천원)'
        ,'Y-1_세전계속사업이익(천원)'
        ,'Y-1_영업이익(천원)'
        ,'Y-1_매출총이익(천원)'
        ,'Y-1_차입금의존도(p)'
        ,'Y-1_당기순이익률(p)'
        ,'Y-1_세전계속사업이익률(p)'
        ,'Y-1_영업이익률(p)'
        ,'Y-1_매출총이익률(p)'
        ,'Y-1_총자산회전율(회)'
        ,'Y-1_총자산증가율(p)'
        ,'Y-1_총자산부채비율(p)'
        ,'Y-2_총자산(천원)'
        ,'Y-2_현금및현금성자산(천원)'
        ,'Y-2_총부채(천원)'
        ,'Y-2_총자본(천원)'
        ,'Y-2_매출액(천원)'
        ,'Y-2_당기순이익(천원)'
        ,'Y-2_세전계속사업이익(천원)'
        ,'Y-2_영업이익(천원)'
        ,'Y-2_매출총이익(천원)'
        ,'Y-2_차입금의존도(p)'
        ,'Y-2_당기순이익률(p)'
        ,'Y-2_세전계속사업이익률(p)'
        ,'Y-2_영업이익률(p)'
        ,'Y-2_매출총이익률(p)'
        ,'Y-2_총자산회전율(회)'
        ,'Y-2_총자산증가율(p)'
        ,'Y-2_총자산부채비율(p)'
        ,'Y-3_총자산(천원)'
        ,'Y-3_현금및현금성자산(천원)'
        ,'Y-3_총부채(천원)'
        ,'Y-3_총자본(천원)'
        ,'Y-3_매출액(천원)'
        ,'Y-3_당기순이익(천원)'
        ,'Y-3_세전계속사업이익(천원)'
        ,'Y-3_영업이익(천원)'
        ,'Y-3_매출총이익(천원)'
        ,'Y-3_차입금의존도(p)'
        ,'Y-3_당기순이익률(p)'
        ,'Y-3_세전계속사업이익률(p)'
        ,'Y-3_영업이익률(p)'
        ,'Y-3_매출총이익률(p)'
        ,'Y-3_총자산회전율(회)'
        ,'Y-3_총자산증가율(p)'
        ,'Y-3_총자산부채비율(p)'
        ,'R1-R2_외국인계매도비율변화'
        ,'R2-R3_외국인계매도비율변화'
        ,'R3-R4_외국인계매도비율변화'
        ,'R1-R2_외국인계매수비율변화'
        ,'R2-R3_외국인계매수비율변화'
        ,'R3-R4_외국인계매수비율변화'
        ,'R1-R2_기관계매도비율변화'
        ,'R2-R3_기관계매도비율변화'
        ,'R3-R4_기관계매도비율변화'
        ,'R1-R2_기관계매수비율변화'
        ,'R2-R3_기관계매수비율변화'
        ,'R3-R4_기관계매수비율변화'
        ,'수익률 (1개월)(p)'
        ,'수익률 (3개월)(p)'
        ,'수익률 (6개월)(p)'
        ,'부도기사비율_m1'
        ,'부도기사비율_m2'
        ,'부도기사비율_m3'
        ,'부도기사비율_m4'
        ,'부도기사비율_m5'
        ,'부도기사비율_m6'
        ,'부도기사비율_m7'
        ,'부도기사비율_m8'
        ,'부도기사비율_m9'
        ,'부도기사비율_m10'
        ,'부도기사비율_m11'
        ,'부도기사비율_m12'
        ,'n_opinion_in_r3'
        ,'m1_oil_price'
        ,'m2_oil_price'
        ,'m3_oil_price'
        ,'m4_oil_price'
        ,'m5_oil_price'
        ,'m6_oil_price'
        ,'m7_oil_price'
        ,'m8_oil_price'
        ,'m9_oil_price'
        ,'m10_oil_price'
        ,'m11_oil_price'
        ,'m12_oil_price'
        ,'m1_oil_price_roc_from_month_year_earlier'
        ,'m2_oil_price_roc_from_month_year_earlier'
        ,'m3_oil_price_roc_from_month_year_earlier'
        ,'m4_oil_price_roc_from_month_year_earlier'
        ,'m5_oil_price_roc_from_month_year_earlier'
        ,'m6_oil_price_roc_from_month_year_earlier'
        ,'m7_oil_price_roc_from_month_year_earlier'
        ,'m8_oil_price_roc_from_month_year_earlier'
        ,'m9_oil_price_roc_from_month_year_earlier'
        ,'m10_oil_price_roc_from_month_year_earlier'
        ,'m11_oil_price_roc_from_month_year_earlier'
        ,'m12_oil_price_roc_from_month_year_earlier'
        ,'m1_CD유통수익률(91일)(p)'
        ,'m2_CD유통수익률(91일)(p)'
        ,'m3_CD유통수익률(91일)(p)'
        ,'m4_CD유통수익률(91일)(p)'
        ,'m5_CD유통수익률(91일)(p)'
        ,'m6_CD유통수익률(91일)(p)'
        ,'m7_CD유통수익률(91일)(p)'
        ,'m8_CD유통수익률(91일)(p)'
        ,'m9_CD유통수익률(91일)(p)'
        ,'m10_CD유통수익률(91일)(p)'
        ,'m11_CD유통수익률(91일)(p)'
        ,'m12_CD유통수익률(91일)(p)'
        ,'m1_국고채(3년)(p)'
        ,'m2_국고채(3년)(p)'
        ,'m3_국고채(3년)(p)'
        ,'m4_국고채(3년)(p)'
        ,'m5_국고채(3년)(p)'
        ,'m6_국고채(3년)(p)'
        ,'m7_국고채(3년)(p)'
        ,'m8_국고채(3년)(p)'
        ,'m9_국고채(3년)(p)'
        ,'m10_국고채(3년)(p)'
        ,'m11_국고채(3년)(p)'
        ,'m12_국고채(3년)(p)'
        ,'m1_원달러환율(매매기준율)'
        ,'m2_원달러환율(매매기준율)'
        ,'m3_원달러환율(매매기준율)'
        ,'m4_원달러환율(매매기준율)'
        ,'m5_원달러환율(매매기준율)'
        ,'m6_원달러환율(매매기준율)'
        ,'m7_원달러환율(매매기준율)'
        ,'m8_원달러환율(매매기준율)'
        ,'m9_원달러환율(매매기준율)'
        ,'m10_원달러환율(매매기준율)'
        ,'m11_원달러환율(매매기준율)'
        ,'m12_원달러환율(매매기준율)'
        ,'y1_gdp_growth_rate'
        ,'y2_gdp_growth_rate'
        ,'y3_gdp_growth_rate'
    ]

    dict_for_insert_into_dataframe = {}

    if year == 3:
        column_for_table_onlyX = column_for_table_onlyX_3
    elif year == 2:
        column_for_table_onlyX = column_for_table_onlyX_2

    for each_key in column_for_table_onlyX:
        dict_for_insert_into_dataframe[each_key] = ''

    query_finance_live = """
        SELECT
        stock_code
        ,"총자산(천원)"
        ,"현금및현금성자산(천원)"
        ,"총부채(천원)"
        ,"총자본(천원)"
        ,"매출액(천원)"
        ,"당기순이익(천원)"
        ,"세전계속사업이익(천원)"
        ,"영업이익(천원)"
        ,"매출총이익(천원)"
        ,"차입금의존도(p)"
        ,"당기순이익률(p)"
        ,"세전계속사업이익률(p)"
        ,"영업이익률(p)"
        ,"매출총이익률(p)"
        ,"총자산회전율(회)"
        ,"총자산증가율(전년동기)(p)"
        ,"총부채(천원)"/"총자산(천원)" AS "총자산부채비율(p)"
        FROM finance_data_1999_2020_raw
        WHERE stock_code = (
            SELECT stock_code
            FROM target_company_list
            WHERE (company_name = %s)
        )
        ORDER BY "회계년" DESC
        LIMIT %s;
    """

    query_sell = """
        SELECT r1.stock_code
                ,r1.R1_외국인계매도비율 - r2.R2_외국인계매도비율 AS "R1-R2_외국인계매도비율변화"
                ,r2.R2_외국인계매도비율 - r3.R3_외국인계매도비율 AS "R2-R3_외국인계매도비율변화"
                ,r3.R3_외국인계매도비율 - r4.R4_외국인계매도비율 AS "R3-R4_외국인계매도비율변화"
                ,r1.R1_기관계매도비율 - r2.R2_기관계매도비율 AS "R1-R2_기관계매도비율변화"
                ,r2.R2_기관계매도비율 - r3.R3_기관계매도비율 AS "R2-R3_기관계매도비율변화"
                ,r3.R3_기관계매도비율 - r4.R4_기관계매도비율 AS "R3-R4_기관계매도비율변화"
        FROM (
            SELECT stock_data.stock_code
            ,COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R1_외국인계매도비율"
            ,COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R1_기관계매도비율"
            FROM stock_data_2000_2020_raw stock_data
            WHERE stock_code = (SELECT stock_code
                                FROM target_company_list
                                WHERE company_name = %s)
            AND stock_data."종가(원)" IS NOT NULL
            AND stock_data.date BETWEEN ('2019-12-31'-30) AND '2019-12-31'
            GROUP BY stock_data.stock_code
        ) r1 INNER JOIN (
            SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R2_외국인계매도비율"
                ,COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R2_기관계매도비율"
                FROM stock_data_2000_2020_raw stock_data
                WHERE stock_code = (SELECT stock_code
                                    FROM target_company_list
                                    WHERE company_name = %s)
                AND stock_data."종가(원)" IS NOT NULL
                AND stock_data.date BETWEEN ('2019-12-31'-60) AND ('2019-12-31'-31)
                GROUP BY stock_data.stock_code
        ) r2 ON r1.stock_code = r2.stock_code
        INNER JOIN (
            SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R3_외국인계매도비율"
                ,COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R3_기관계매도비율"
                FROM stock_data_2000_2020_raw stock_data
                WHERE stock_code = (SELECT stock_code
                                    FROM target_company_list
                                    WHERE company_name = %s)
                AND stock_data."종가(원)" IS NOT NULL
                AND stock_data.date BETWEEN ('2019-12-31'-90) AND ('2019-12-31'-61)
                GROUP BY stock_data.stock_code
        ) r3 ON r1.stock_code = r3.stock_code
        INNER JOIN (
            SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R4_외국인계매도비율"
                ,COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매도수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매도수량(개인)(주)"),0)+COALESCE(sum(stock_data."매도수량(기관계)(주)"),0)) as "R4_기관계매도비율"
                FROM stock_data_2000_2020_raw stock_data
                WHERE stock_code = (SELECT stock_code
                                    FROM target_company_list
                                    WHERE company_name = %s)
                AND stock_data."종가(원)" IS NOT NULL
                AND stock_data.date BETWEEN ('2019-12-31'-120) AND ('2019-12-31'-91)
                GROUP BY stock_data.stock_code
        ) r4 ON r1.stock_code = r4.stock_code;
    """

    query_buy = """
        SELECT r1.stock_code
            ,r1.R1_외국인계매수비율 - r2.R2_외국인계매수비율 AS "R1-R2_외국인계매수비율변화"
            ,r2.R2_외국인계매수비율 - r3.R3_외국인계매수비율 AS "R2-R3_외국인계매수비율변화"
            ,r3.R3_외국인계매수비율 - r4.R4_외국인계매수비율 AS "R3-R4_외국인계매수비율변화"
            ,r1.R1_기관계매수비율 - r2.R2_기관계매수비율 AS "R1-R2_기관계매수비율변화"
            ,r2.R2_기관계매수비율 - r3.R3_기관계매수비율 AS "R2-R3_기관계매수비율변화"
            ,r3.R3_기관계매수비율 - r4.R4_기관계매수비율 AS "R3-R4_기관계매수비율변화"
        FROM (
            SELECT stock_data.stock_code
            ,COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R1_외국인계매수비율"
            ,COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R1_기관계매수비율"
            FROM stock_data_2000_2020_raw stock_data
            WHERE stock_code = (SELECT stock_code
                                FROM target_company_list
                                WHERE company_name = %s)
            AND stock_data."종가(원)" IS NOT NULL
            AND stock_data.date BETWEEN ('2019-12-31'-30) AND '2019-12-31'
            GROUP BY stock_data.stock_code
        ) r1 INNER JOIN (
            SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R2_외국인계매수비율"
                ,COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R2_기관계매수비율"
                FROM stock_data_2000_2020_raw stock_data
                WHERE stock_code = (SELECT stock_code
                                    FROM target_company_list
                                    WHERE company_name = %s)
                AND stock_data."종가(원)" IS NOT NULL
                AND stock_data.date BETWEEN ('2019-12-31'-60) AND ('2019-12-31'-31)
                GROUP BY stock_data.stock_code
        ) r2 ON r1.stock_code = r2.stock_code
        INNER JOIN (
            SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R3_외국인계매수비율"
                ,COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R3_기관계매수비율"
                FROM stock_data_2000_2020_raw stock_data
                WHERE stock_code = (SELECT stock_code
                                    FROM target_company_list
                                    WHERE company_name = %s)
                AND stock_data."종가(원)" IS NOT NULL
                AND stock_data.date BETWEEN ('2019-12-31'-90) AND ('2019-12-31'-61)
                GROUP BY stock_data.stock_code
        ) r3 ON r1.stock_code = r3.stock_code
        INNER JOIN (
            SELECT stock_data.stock_code
                ,COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R4_외국인계매수비율"
                ,COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)/(COALESCE(sum(stock_data."매수수량(외국인계)(주)"),0)+COALESCE(sum(stock_data."매수수량(개인)(주)"),0)+COALESCE(sum(stock_data."매수수량(기관계)(주)"),0)) as "R4_기관계매수비율"
                FROM stock_data_2000_2020_raw stock_data
                WHERE stock_code = (SELECT stock_code
                                    FROM target_company_list
                                    WHERE company_name = %s)
                AND stock_data."종가(원)" IS NOT NULL
                AND stock_data.date BETWEEN ('2019-12-31'-120) AND ('2019-12-31'-91)
                GROUP BY stock_data.stock_code
        ) r4 ON r1.stock_code = r4.stock_code;
    """

    query_earnings_rate = """
        SELECT stock_code, "수익률 (1개월)(p)", "수익률 (3개월)(p)", "수익률 (6개월)(p)"
        FROM stock_data_2000_2020_raw
        WHERE stock_code = (SELECT stock_code
                            FROM target_company_list
                            WHERE company_name = %s)
        AND date = '2019-12-31';
    """

    query_article_ratio = """
        SELECT *
        FROM bankruptcy_article_ratio_table_live
        WHERE stock_code = (
            SELECT stock_code
            FROM target_company_list
            WHERE (company_name = %s)
        );
    """

    query_audit_report = """
        SELECT n_opinion FROM dart_audit_report_data
        WHERE stock_code = (
            SELECT stock_code
            FROM target_company_list
            WHERE (company_name = %s)
        )
        ORDER BY report_date DESC NULLS LAST
        LIMIT 3;
    """

    query_oil = """
        SELECT "Dubai(dollar)"
        FROM oil_price_dubai_month
        WHERE ((date_trunc('month', DATEADD(week,1,DATEADD(month,1,TO_DATE(year_month, 'YYYY-MM')))))::date <= add_months(%s,%s))
        AND ((date_trunc('month', DATEADD(week,1,DATEADD(month,1,TO_DATE(year_month, 'YYYY-MM')))))::date > add_months(%s,%s));
    """

    query_oil_price_roc = """
        SELECT "전년동월대비_가격증감율"
        FROM oil_price_dubai_month
        WHERE ((date_trunc('month', DATEADD(week,1,DATEADD(month,1,TO_DATE(year_month, 'YYYY-MM')))))::date <= add_months(%s,%s))
        AND ((date_trunc('month', DATEADD(week,1,DATEADD(month,1,TO_DATE(year_month, 'YYYY-MM')))))::date > add_months(%s,%s));
    """

    query_cd = """
        SELECT "CD유통수익률(91일)(p)"
        FROM marcoeco_data_from_bok
        WHERE ((date_trunc('month', DATEADD(week,1,DATEADD(month,1,TO_DATE(year_month, 'YYYY-MM')))))::date <= add_months(%s,%s))
        AND ((date_trunc('month', DATEADD(week,1,DATEADD(month,1,TO_DATE(year_month, 'YYYY-MM')))))::date > add_months(%s,%s));
    """

    query_bond = """
        SELECT "국고채(3년)(p)"
        FROM marcoeco_data_from_bok
        WHERE ((date_trunc('month', DATEADD(week,1,DATEADD(month,1,TO_DATE(year_month, 'YYYY-MM')))))::date <= add_months(%s,%s))
        AND ((date_trunc('month', DATEADD(week,1,DATEADD(month,1,TO_DATE(year_month, 'YYYY-MM')))))::date > add_months(%s,%s));
    """

    query_exchange_rate = """
        SELECT "원달러환율(매매기준율)"
        FROM marcoeco_data_from_bok
        WHERE ((date_trunc('month', DATEADD(week,1,DATEADD(month,1,TO_DATE(year_month, 'YYYY-MM')))))::date <= add_months(%s,%s))
        AND ((date_trunc('month', DATEADD(week,1,DATEADD(month,1,TO_DATE(year_month, 'YYYY-MM')))))::date > add_months(%s,%s));
    """

    query_gdp_growth_rate = """
        SELECT gdp_growth_rate
        FROM gdp_growth_rate
        WHERE year = TO_CHAR(DATEADD(year,%s,%s), 'YYYY');
    """

    with psycopg2.connect(**connect_param) as con:
        with con.cursor() as cur:
            cur.execute(query_finance_live, [company_name, year])
            for idx, each_row in enumerate(cur):
                if idx == 0:
                    dict_for_insert_into_dataframe['stock_code'] = each_row[0]
                    dict_for_insert_into_dataframe["Y-1_총자산(천원)"] = each_row[1]
                    dict_for_insert_into_dataframe["Y-1_현금및현금성자산(천원)"] = each_row[2]
                    dict_for_insert_into_dataframe["Y-1_총부채(천원)"] = each_row[3]
                    dict_for_insert_into_dataframe["Y-1_총자본(천원)"] = each_row[4]
                    dict_for_insert_into_dataframe["Y-1_매출액(천원)"] = each_row[5]
                    dict_for_insert_into_dataframe["Y-1_당기순이익(천원)"] = each_row[6]
                    dict_for_insert_into_dataframe["Y-1_세전계속사업이익(천원)"] = each_row[7]
                    dict_for_insert_into_dataframe["Y-1_영업이익(천원)"] = each_row[8]
                    dict_for_insert_into_dataframe["Y-1_매출총이익(천원)"] = each_row[9]
                    dict_for_insert_into_dataframe["Y-1_차입금의존도(p)"] = each_row[10]
                    dict_for_insert_into_dataframe["Y-1_당기순이익률(p)"] = each_row[11]
                    dict_for_insert_into_dataframe["Y-1_세전계속사업이익률(p)"] = each_row[12]
                    dict_for_insert_into_dataframe["Y-1_영업이익률(p)"] = each_row[13]
                    dict_for_insert_into_dataframe["Y-1_매출총이익률(p)"] = each_row[14]
                    dict_for_insert_into_dataframe["Y-1_총자산회전율(회)"] = each_row[15]
                    dict_for_insert_into_dataframe["Y-1_총자산증가율(p)"] = each_row[16]
                    dict_for_insert_into_dataframe["Y-1_총자산부채비율(p)"] = each_row[17]
                elif idx == 1:
                    dict_for_insert_into_dataframe["Y-2_총자산(천원)"] = each_row[1]
                    dict_for_insert_into_dataframe["Y-2_현금및현금성자산(천원)"] = each_row[2]
                    dict_for_insert_into_dataframe["Y-2_총부채(천원)"] = each_row[3]
                    dict_for_insert_into_dataframe["Y-2_총자본(천원)"] = each_row[4]
                    dict_for_insert_into_dataframe["Y-2_매출액(천원)"] = each_row[5]
                    dict_for_insert_into_dataframe["Y-2_당기순이익(천원)"] = each_row[6]
                    dict_for_insert_into_dataframe["Y-2_세전계속사업이익(천원)"] = each_row[7]
                    dict_for_insert_into_dataframe["Y-2_영업이익(천원)"] = each_row[8]
                    dict_for_insert_into_dataframe["Y-2_매출총이익(천원)"] = each_row[9]
                    dict_for_insert_into_dataframe["Y-2_차입금의존도(p)"] = each_row[10]
                    dict_for_insert_into_dataframe["Y-2_당기순이익률(p)"] = each_row[11]
                    dict_for_insert_into_dataframe["Y-2_세전계속사업이익률(p)"] = each_row[12]
                    dict_for_insert_into_dataframe["Y-2_영업이익률(p)"] = each_row[13]
                    dict_for_insert_into_dataframe["Y-2_매출총이익률(p)"] = each_row[14]
                    dict_for_insert_into_dataframe["Y-2_총자산회전율(회)"] = each_row[15]
                    dict_for_insert_into_dataframe["Y-2_총자산증가율(p)"] = each_row[16]
                    dict_for_insert_into_dataframe["Y-2_총자산부채비율(p)"] = each_row[17]
                elif idx == 2:
                    dict_for_insert_into_dataframe["Y-3_총자산(천원)"] = each_row[1]
                    dict_for_insert_into_dataframe["Y-3_현금및현금성자산(천원)"] = each_row[2]
                    dict_for_insert_into_dataframe["Y-3_총부채(천원)"] = each_row[3]
                    dict_for_insert_into_dataframe["Y-3_총자본(천원)"] = each_row[4]
                    dict_for_insert_into_dataframe["Y-3_매출액(천원)"] = each_row[5]
                    dict_for_insert_into_dataframe["Y-3_당기순이익(천원)"] = each_row[6]
                    dict_for_insert_into_dataframe["Y-3_세전계속사업이익(천원)"] = each_row[7]
                    dict_for_insert_into_dataframe["Y-3_영업이익(천원)"] = each_row[8]
                    dict_for_insert_into_dataframe["Y-3_매출총이익(천원)"] = each_row[9]
                    dict_for_insert_into_dataframe["Y-3_차입금의존도(p)"] = each_row[10]
                    dict_for_insert_into_dataframe["Y-3_당기순이익률(p)"] = each_row[11]
                    dict_for_insert_into_dataframe["Y-3_세전계속사업이익률(p)"] = each_row[12]
                    dict_for_insert_into_dataframe["Y-3_영업이익률(p)"] = each_row[13]
                    dict_for_insert_into_dataframe["Y-3_매출총이익률(p)"] = each_row[14]
                    dict_for_insert_into_dataframe["Y-3_총자산회전율(회)"] = each_row[15]
                    dict_for_insert_into_dataframe["Y-3_총자산증가율(p)"] = each_row[16]
                    dict_for_insert_into_dataframe["Y-3_총자산부채비율(p)"] = each_row[17]

        with psycopg2.connect(**connect_param) as con:
            with con.cursor() as cur:
                cur.execute(query_sell, [company_name, company_name, company_name, company_name])
                for each in cur:
                    dict_for_insert_into_dataframe["R1-R2_외국인계매도비율변화"] = each[1]
                    dict_for_insert_into_dataframe["R2-R3_외국인계매도비율변화"] = each[2]
                    dict_for_insert_into_dataframe["R3-R4_외국인계매도비율변화"] = each[3]
                    dict_for_insert_into_dataframe["R1-R2_기관계매도비율변화"] = each[4]
                    dict_for_insert_into_dataframe["R2-R3_기관계매도비율변화"] = each[5]
                    dict_for_insert_into_dataframe["R3-R4_기관계매도비율변화"] = each[6]

        with psycopg2.connect(**connect_param) as con:
            with con.cursor() as cur:           
                cur.execute(query_buy, [company_name, company_name, company_name, company_name])
                for each in cur:
                    dict_for_insert_into_dataframe["R1-R2_외국인계매수비율변화"] = each[1]
                    dict_for_insert_into_dataframe["R2-R3_외국인계매수비율변화"] = each[2]
                    dict_for_insert_into_dataframe["R3-R4_외국인계매수비율변화"] = each[3]
                    dict_for_insert_into_dataframe["R1-R2_기관계매수비율변화"] = each[4]
                    dict_for_insert_into_dataframe["R2-R3_기관계매수비율변화"] = each[5]
                    dict_for_insert_into_dataframe["R3-R4_기관계매수비율변화"] = each[6]

    with psycopg2.connect(**connect_param) as con:
        with con.cursor() as cur:       
            cur.execute(query_earnings_rate, [company_name])
            for each in cur:
                dict_for_insert_into_dataframe['수익률 (1개월)(p)'] = each[1]
                dict_for_insert_into_dataframe['수익률 (3개월)(p)'] = each[2]
                dict_for_insert_into_dataframe['수익률 (6개월)(p)'] = each[3]

    with psycopg2.connect(**connect_param) as con:
        with con.cursor() as cur:      
            cur.execute(query_article_ratio, [company_name])
            for each in cur:
                dict_for_insert_into_dataframe['부도기사비율_m1'] = each[1]
                dict_for_insert_into_dataframe['부도기사비율_m2'] = each[2]
                dict_for_insert_into_dataframe['부도기사비율_m3'] = each[3]
                dict_for_insert_into_dataframe['부도기사비율_m4'] = each[4]
                dict_for_insert_into_dataframe['부도기사비율_m5'] = each[5]
                dict_for_insert_into_dataframe['부도기사비율_m6'] = each[6]
                dict_for_insert_into_dataframe['부도기사비율_m7'] = each[7]
                dict_for_insert_into_dataframe['부도기사비율_m8'] = each[8]
                dict_for_insert_into_dataframe['부도기사비율_m9'] = each[9]
                dict_for_insert_into_dataframe['부도기사비율_m10'] = each[10]
                dict_for_insert_into_dataframe['부도기사비율_m11'] = each[11]
                dict_for_insert_into_dataframe['부도기사비율_m12'] = each[12]

    with psycopg2.connect(**connect_param) as con:
        with con.cursor() as cur:    
            cur.execute(query_audit_report, [company_name])
            list_for_audit_check = []
            for each in cur:
                list_for_audit_check.append(each[0])
            dict_for_insert_into_dataframe['n_opinion_in_r3'] = True in list_for_audit_check 

    for each_idx in range(0,12):
        with psycopg2.connect(**connect_param) as con:
            with con.cursor() as cur:
                cur.execute(query_oil, ('2019-12-31', 0-each_idx, '2019-12-31', -1-each_idx))
                for each in cur:
                    dict_for_insert_into_dataframe[column_for_table_onlyX[63 + 17*(year-2) + each_idx]] = each[0]
        with psycopg2.connect(**connect_param) as con:
            with con.cursor() as cur:                    
                cur.execute(query_oil_price_roc, ('2019-12-31', 0-each_idx, '2019-12-31', -1-each_idx))
                for each in cur:
                    dict_for_insert_into_dataframe[column_for_table_onlyX[75 + 17*(year-2) + each_idx]] = each[0]
        with psycopg2.connect(**connect_param) as con:
            with con.cursor() as cur:                
                cur.execute(query_cd, ('2019-12-31', 0-each_idx, '2019-12-31', -1-each_idx))
                for each in cur:
                    dict_for_insert_into_dataframe[column_for_table_onlyX[87 + 17*(year-2) + each_idx]] = each[0]
        with psycopg2.connect(**connect_param) as con:
            with con.cursor() as cur:                    
                cur.execute(query_bond, ('2019-12-31', 0-each_idx, '2019-12-31', -1-each_idx))
                for each in cur:
                    dict_for_insert_into_dataframe[column_for_table_onlyX[99 + 17*(year-2) + each_idx]] = each[0]
        with psycopg2.connect(**connect_param) as con:
            with con.cursor() as cur:                    
                cur.execute(query_exchange_rate, ('2019-12-31', 0-each_idx, '2019-12-31', -1-each_idx))
                for each in cur:
                    dict_for_insert_into_dataframe[column_for_table_onlyX[111 + 17*(year-2) + each_idx]] = each[0]

        for each_idx in range(0,3):
            with psycopg2.connect(**connect_param) as con:
                with con.cursor() as cur:
                        cur.execute(query_gdp_growth_rate, (-1-each_idx, '2019-12-31'))
                        for each in cur:
                            dict_for_insert_into_dataframe[column_for_table_onlyX[123 + 17*(year-2) + each_idx]] = each[0]


    live_dataframe = pd.DataFrame(dict_for_insert_into_dataframe, index = [0])
    live_dataframe.set_index('stock_code', inplace=True)

    with open('best_hp_tuned_model_6m.pickle', 'rb') as f:
        best_model_6m = pickle.load(f)

    with open('best_hp_tuned_model_3m.pickle', 'rb') as f:
        best_model_3m = pickle.load(f)

    with open('best_hp_tuned_model_1y.pickle', 'rb') as f:
        best_model_1y = pickle.load(f)

    result_6m = best_model_6m.predict(live_dataframe)
    proba_6m = best_model_6m.predict_proba(live_dataframe)

    result_3m = best_model_3m.predict(live_dataframe)
    proba_3m = best_model_3m.predict_proba(live_dataframe)

    result_1y = best_model_1y.predict(live_dataframe)
    proba_1y = best_model_1y.predict_proba(live_dataframe)

    data = {'result_6m':str(result_6m[0]), 'proba_6m_false':round(proba_6m[0][0]*100,2), 'proba_6m_true':round(proba_6m[0][1]*100,2)
            ,'result_3m':str(result_3m[0]), 'proba_3m_false':round(proba_3m[0][0]*100,2), 'proba_3m_true':round(proba_3m[0][1]*100,2)
            ,'result_1y':str(result_1y[0]), 'proba_1y_false':round(proba_1y[0][0]*100,2), 'proba_1y_true':round(proba_1y[0][1]*100,2)
    }

    return data

In [193]:
model_prediction('LG전자', year=2)

{'result_6m': 'False',
 'proba_6m_false': 97.09,
 'proba_6m_true': 2.91,
 'result_3m': 'False',
 'proba_3m_false': 95.53,
 'proba_3m_true': 4.47,
 'result_1y': 'False',
 'proba_1y_false': 96.49,
 'proba_1y_true': 3.51}